In [3]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
#WHERE WE EXTRACT TABLE NAME FROM CSV
from bs4 import BeautifulSoup
from tika import parser
import os
import re
import camelot
from fuzzywuzzy import fuzz
from datetime import datetime
import textwrap

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
#######################FUNCTIONS##################################################
def get_table_titles(page:int,file,df:pd.DataFrame) -> list():#pd.DataFrame:
    tbl_names_trunc = list()
    tbl_names = list(df[(df['page_number']== page) & (df['DataID_pdf']== file)]['final_table_title'])
    for tbl in tbl_names:
        if len(tbl) > 218:
            tbl_names_trunc.append(textwrap.shorten(tbl,width = 218))
        else:
            tbl_names_trunc.append(tbl)      
    return tbl_names_trunc
#********************************************************************************#
def replace_chars(text:str) -> str:
    chars_0 = ['\n',':']
    chars_1 = ['/','\\']
    for c in chars_0:
        text = text.replace(c, ' ')
    for cc in chars_1:
        text = text.replace(cc,'_')
    return text
#********************************************************************************#
def replace_chars_strings(lst:list) -> list:
    new_lst = []
    for itm in lst:
        new_lst.append(replace_chars(itm))       
    return new_lst



In [6]:
#Main Function
def extract_tables(file:str(),df:pd.DataFrame) -> list:   
    failed_pdf = []
    start_time = datetime.now()

    file_path = 'F:/Environmental Baseline Data/Version 4 - Final/PDF/{}'.format(file)

    file_name = file_path.split('/')[-1].replace('.pdf','')

    try:
        data = parser.from_file(file_path,xmlContent=True)
        #raw_xml = parser.from_file('A6T2V6.pdf', xmlContent=True)

        #xml tag <div> splitting point for pages
        soup = BeautifulSoup(data['content'], 'lxml')
        pages = soup.find_all('div', attrs={'class': 'page'})

        title_dict = dict()
        start_time = datetime.now()
        for ind, page in enumerate (pages):
            pg_num = ind
            chars = []
            #camelot table objects for each page of the pdf
            try:
                tables = camelot.read_pdf(file_path, pages = str(pg_num+1), flag_size=True, copy_text=['v'],strip_text = '\n',line_scale=40, f = 'csv',flavour = 'stream')  #loop len(tables)
            except:
                continue
            #get table names in page == pg_num by parsing get_table_titles() function
            title_lst_raw = get_table_titles(pg_num+1,file,df)
            title_lst = replace_chars_strings(title_lst_raw)
            #get total number of table objects detected by Camelot in page == pg_num
            tb_num = tables.n

            #VIEW
            print(title_lst)  
            #if Camelot returns NO table on the page continue the loop and go to the next page
            if tb_num == 0:
                print("No table on page "+ str(pg_num+1) + " is detected")
                continue
            #if whitespace of the detected table is larger than 69% of the entire table and there is only
            #one table on that page, identify this as figure and continute the loop and go to the next page
            elif tb_num == 1 and (tables[0].parsing_report)['whitespace'] > 69.0:
                print("Page {} contains an image".format((tables[0].parsing_report)['page'] ))  
                continue
            #in case only one table is present on the page,
            elif tb_num == 1:
                #this block distills the dataframe with proper column names
                df_tb = tables[0].df
                df_tb = df_tb.replace('/na', '_', regex = True)
                colname = df_tb.iloc[0].str.replace('\n',' ',regex=True)
                df_tb.columns = colname
                df_tb = df_tb[1:]   
                #df_tb = df_tb.iloc[1:]
                #in case no title is extracted from this page but we know that there is one table
                if len(title_lst) == 0:
                    #let's say we are on page number x and this if statement checks whether page number x-1 contianed a table or not
                    #if the result is TRUE we assign the title of last table on previous page to this page's title-less table
                    if (pg_num-1 in title_dict):
                        #find the list of tables on the previous page
                        lst_tbl = (title_dict.get(pg_num-1))[-1]
                        lst_tbl_df = lst_tbl[1]
                        #find similarity score of column names of table on page x and page x-1
                        col_concat_curr = list(df_tb.columns.values)
                        ccc = ''.join(col_concat_curr)
                        col_join_curr = (ccc.replace(' ','')).replace('\n','')
                        col_concat_prev = list(lst_tbl_df.columns.values)
                        ccp = ''.join(col_concat_prev)
                        col_join_prev = (ccp.replace(' ','')).replace('\n','')  
                        ratio_similarity = fuzz.token_sort_ratio(col_join_curr, col_join_prev)
                        #check if the columns of the last table on the previous page are the same as the table on this page
                        if (len((set(lst_tbl_df.columns)).difference(set(df_tb.columns))) == 0) or (len(set(lst_tbl_df.columns))== len(set(df_tb.columns))) or (ratio_similarity > 89):
                            xl_name = lst_tbl[2]
                            chars.append([0,df_tb,xl_name])
                            xlsx_name = file_name + '_' + xl_name +'_'+str(pg_num+1)+'_'+str(1)+ '.csv'
                            df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')     
                        else:
                            xl_name = file_name
                            chars.append([0,df_tb,xl_name])
                            xlsx_name = file_name + '_' +str(pg_num+1)+'_'+str(1)+ '.csv'
                            df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')
                        title_dict[pg_num] = chars

                    else:
                        xl_name = file_name       
                        chars.append([0,df_tb,xl_name])
                        xlsx_name = file_name + '_' +str(pg_num+1)+'_'+str(1)+ '.csv'
                        df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')
                        title_dict[pg_num] = chars
                else:
                    xl_name = title_lst[0]
        #                xl_name = xl_name.replace('/','_')
        #                xl_name = xl_name.replace(':','')
                    #store page number, index of the table, and its name in a dictionary
                    chars.append([0,df_tb,xl_name])
                    xlsx_name = file_name + '_' + xl_name +'_'+str(pg_num+1)+'_'+str(1)+ '.csv'
                    df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')
                    title_dict[pg_num] = chars

            else:
                if len(title_lst) >= tb_num :
                    for j in range(0,tb_num):
                        df_tb = tables[j].df
                        df_tb = df_tb.replace('/na', '_', regex = True)
                        df_tb.columns = df_tb.iloc[0].str.replace('\n',' ',regex=True)
                        df_tb = df_tb[1:]
                        #df_tb = df_tb.iloc[1:]
                        xl_name = title_lst[j]
        #                    xl_name = xl_name.replace('/','_')
        #                    xl_name = xl_name.replace(':','')

                        #store page number, index of the table, and its name in a dictionary
                        chars.append([j,df_tb,xl_name])

                        xlsx_name = file_name + '_' + xl_name +'_'+str(pg_num+1)+'_'+str(j+1)+ '.csv'
                        df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')

                    title_dict[pg_num] = chars

                elif len(title_lst) < tb_num :
                    try:
                        #first case: if the first table in the page is conitnuos of what was on the previous page
                        df_tb = tables[0].df
                        df_tb = df_tb.replace('/na', '_', regex = True)
                        df_tb.columns = df_tb.iloc[0].str.replace('\n',' ',regex=True)
                        df_tb = df_tb[1:]
                        #df_tb = df_tb.iloc[1:]

                        if (pg_num-1 in title_dict):
                            #find the list of tables on the previous page
                            lst_tbl = (title_dict.get(pg_num-1))[-1]
                            lst_tbl_df = lst_tbl[1]
                            #check if the columns of the last table on the previous page are the same as the table on this page

                            col_concat_curr = list(df_tb.columns.values)
                            ccc = ''.join(col_concat_curr)
                            col_join_curr = (ccc.replace(' ','')).replace('\n','')
                            col_concat_prev = list(lst_tbl_df.columns.values)
                            ccp = ''.join(col_concat_prev)
                            col_join_prev = (ccp.replace(' ','')).replace('\n','')  
                            ratio_similarity = fuzz.token_sort_ratio(ccc, ccp)

                            if len((set(lst_tbl_df.columns)).difference(set(df_tb.columns))) == 0 or len(set(lst_tbl_df.columns))== len(set(df_tb.columns)) or ratio_similarity > 89:
                                xl_name = lst_tbl[2]                           
                                chars.append([0,df_tb,xl_name])                         
                                xlsx_name = file_name + '_' + xl_name +'_'+str(pg_num+1)+'_'+str(1)+ '.csv'
                                df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')

                            else:
                                xl_name = file_name   
                                chars.append([0,df_tb,xl_name])
                                xlsx_name = file_name + '_' +str(pg_num+1)+'_'+str(1)+ '.csv'
                                df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')
                        else:
                            xl_name = file_name       
                            chars.append([0,df_tb,xl_name])
                            xlsx_name = file_name + '_' +str(pg_num+1)+'_'+str(1)+ '.csv'
                            df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')

                        indx = 0
                        for j in range(1,len(title_lst)+1):
                            df_tb = tables[j].df
                            df_tb = df_tb.replace('/na', '_', regex = True)
                            df_tb.columns = df_tb.iloc[0].str.replace('\n',' ',regex=True)
                            df_tb = df_tb[1:]
                            #df_tb = df_tb.iloc[1:]
                            xl_name = title_lst[indx]
                            indx = indx + 1
        #                        xl_name = xl_name.replace('/','_')
        #                        xl_name = xl_name.replace(':','')
                            #store page number, index of the table, and its name in a dictionary
                            chars.append([j,df_tb,xl_name])

                            xlsx_name = file_name + '_' + xl_name +'_'+str(pg_num+1)+'_'+str(j+1)+ '.csv'
                            #xlsx_name = z.split('\\')[-1] + '-' + str(pg[i]+1) + str(j) + '.xlsx'
                            df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')

                        for j in range(len(title_lst)+1,tb_num):
                            df_tb = tables[j].df
                            df_tb = df_tb.replace('/na', '_', regex = True)
                            df_tb.columns = df_tb.iloc[0].str.replace('\n',' ',regex=True)
                            df_tb = df_tb[1:]
                            #df_tb = df_tb.iloc[1:]
                            xl_name = file_name
                            #store page number, index of the table, and its name in a dictionary
                            chars.append([j,df_tb,xl_name])                        

                            xlsx_name = file_name + '_' + xl_name +'_'+str(pg_num+1)+'_'+str(j+1)+ '.csv'
                            #xlsx_name = z.split('\\')[-1] + '-' + str(pg[i]+1) + str(j) + '.xlsx'
                            df_tb.to_csv(xlsx_name, index = False, encoding='utf-8-sig')                    
                        title_dict[pg_num] = chars

                    except:
                        print("Function failed on page {}".format(pg_num+1))
                        pass
    except:
        failed_pdf.append(file)
        print("file {} failed to open".format(file))
        pass
        
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time)) 
    
    return failed_pdf

In [7]:
path = 'F:/Environmental Baseline Data/Version 4 - Final/Support files/Table titles raw data/final_table_titles2.csv'
df = pd.read_csv(path, usecols = ['page_number','final_table_title', 'Application title short', 'DataID_pdf', \
                                  'categories', 'Category'])
df = df[df['categories'] > 0] 
df = df[df['Category'] == 'Table']
df['final_table_title'] = df['final_table_title'].str.title()
df.head()

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
2,8,Table 1 Hydrometric Data From Discharge Statio...,0.75,1059614.pdf,Application for North Montney Project,Table
3,9,Table 2 Fish Species That May Occur In The Upp...,0.75,1059614.pdf,Application for North Montney Project,Table
4,14,Table 3 Summary Of Aquatics Field Work And Abo...,0.75,1059614.pdf,Application for North Montney Project,Table
5,17,Table 4 Summary Of Watercourse Crossings Along...,0.75,1059614.pdf,Application for North Montney Project,Table
6,18,Table 4 Summary Of Watercourse Crossings Along...,0.35,1059614.pdf,Application for North Montney Project,Table


In [8]:
df['Application title short'].unique().tolist()

['Application for North Montney Project',
 'Application for Trans Mountain Expansion Project',
 'Application for the Wolverine River Lateral Loop Carmon Creek Section',
 'Applications for Energy East, Asset Transfer and Eastern Mainline  Eastern Mainline ESA',
 'Application for the Line 3 Replacement Program',
 'Application to Construct and Operate Ekwan Pipeline',
 'Application for the 2017 NGTL System Expansion',
 'Application for the Towerbirch Expansion Project',
 'Application for the Albersun Pipeline Asset Purchase G',
 'Applications for Energy East, Asset Transfer and Eastern Mainline',
 'Application for the Wyndwood Pipeline Expansion Project',
 'Application for the Spruce Ridge Program',
 'Application for the construction of the West Path Delivery Project',
 'Application for the Goldboro Gas Plant and 26" Gathering Pipeline Abandonment',
 'Application for 2021 NGTL System Expansion Project',
 'Application for Leave to Abandon Deep Panuke Pipeline',
 'Application for the Constr

## Application for 2021 NGTL System Expansion Project

In [ ]:
hearing = 'Application for 2021 NGTL System Expansion Project' 
ngtl2021 = df[df['Application title short'] == hearing].reset_index(drop = True)
ngtl2021.head()

### Change this folder to the path were you want the tables saved

In [ ]:
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\ngtl2021')

### Change the dataframe name accordingly

In [ ]:
files = list(ngtl2021['DataID_pdf'].unique())
files

### call the main function -- pass filename and dataframe as function arguments

In [ ]:
for file in files:
    extract_tables(file,ngtl2021)


## Application for the 2017 NGTL System Expansion

In [ ]:
hearing = 'Application for the 2017 NGTL System Expansion'
ngtl2017 = df[df['Application title short'] == hearing].reset_index(drop = True)
ngtl2017.head()

### Change this folder to the path were you want the tables saved

In [ ]:
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\ngtl2017')

### Change the dataframe name accordingly

In [ ]:
files = list(ngtl2017['DataID_pdf'].unique())
files

### call the main function -- pass filename and dataframe as function arguments

In [ ]:
for file in files:
    extract_tables(file,ngtl2017)

## Application for Northwest Mainline Komie North Extension

In [7]:
hearing = 'Application for Northwest Mainline Komie North Extension'
komie_north = df[df['Application title short'] == hearing].reset_index(drop = True)
komie_north.head()

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,7,Table 1 Summary Of Identified Residual Project...,1.0,729032.pdf,Application for Northwest Mainline Komie North...,Table
1,9,Table 1 Summary Of Identified Residual Project...,1.0,729032.pdf,Application for Northwest Mainline Komie North...,Table
2,11,Table 1 Summary Of Identified Residual Project...,1.0,729032.pdf,Application for Northwest Mainline Komie North...,Table
3,13,Table 1 Summary Of Identified Residual Project...,1.0,729032.pdf,Application for Northwest Mainline Komie North...,Table
4,15,Table 1 Summary Of Identified Residual Project...,1.0,729032.pdf,Application for Northwest Mainline Komie North...,Table


### Change this folder to the path were you want the tables saved

In [8]:
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\komie_north')

### Change the dataframe name accordingly

In [9]:
files = list(komie_north['DataID_pdf'].unique())
files

['729032.pdf',
 '729035.pdf',
 '729038.pdf',
 '729133.pdf',
 '729136.pdf',
 '729139.pdf',
 '729224.pdf',
 '729227.pdf',
 '729230.pdf',
 '729233.pdf',
 '729236.pdf',
 '729247.pdf',
 '729250.pdf',
 '729253.pdf',
 '729256.pdf',
 '729346.pdf',
 '729349.pdf']

### call the main function -- pass filename and dataframe as function arguments

In [ ]:
for file in files:
    extract_tables(file,komie_north)

[]
No table on page 1 is detected


[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
['Table 1 Summary Of Identified Residual Project Effects']


[]
No table on page 8 is detected
['Table 1 Summary Of Identified Residual Project Effects (Cont’D)']


[]
No table on page 10 is detected
['Table 1 Summary Of Identified Residual Project Effects (Cont’D)']


[]
No table on page 12 is detected
['Table 1 Summary Of Identified Residual Project Effects (Cont’D)']


...

[]
No table on page 126 is detected
['Table 3-3 Consultation Activities With Local Contacts And Nongovernment Organizations For The Northwest Mainline Komie North Extension Project (Cont’D)']


[]
No table on page 128 is detected
['Table 3-3 Consultation Activities With Local Contacts And Nongovernment Organizations For The Northwest Mainline Komie North Extension Project (Cont’D)']


[]
No table on page 130 is detected
['Table 3-3 Consultation Activities With Local Contacts And Nongovernment Organizations For The Northwest Mainline Komie North Extension Project (Cont’D)']


[]
No table on page 132 is detected
['Table 3-3 Consultation Activities With Local Contacts And Nongovernment Organizations For The Northwest Mainline Komie North Extension Project (Cont’D)']


[]
No table on page 134 is detected
['Table 3-3 Consultation Activities With Local Contacts And Nongovernment Organizations For The Northwest Mainline Komie North Extension Project (Cont’D)']
[]
No table on page 136 is detected
[]
No table on page 137 is detected
[]
No table on page 138 is detected
[]
No table on page 139 is detected


## Application for the Edmonton to Hardisty Pipeline Project

In [13]:
hearing = 'Application for the Edmonton to Hardisty Pipeline Project'
edmonton_hardisty = df[df['Application title short'] == hearing].reset_index(drop = True)
edmonton_hardisty.head()

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,2,Table 2.1 Technical Details – Edmonton To Hard...,0.75,894827.pdf,Application for the Edmonton to Hardisty Pipel...,Table
1,5,Table 2.2 Pipeline Construction Activities,0.75,894827.pdf,Application for the Edmonton to Hardisty Pipel...,Table
2,6,Table 2.2 Pipeline Construction Activities,0.35,894827.pdf,Application for the Edmonton to Hardisty Pipel...,Table
3,6,Table 2.3 Construction Activities For Pump Sta...,0.75,894827.pdf,Application for the Edmonton to Hardisty Pipel...,Table
4,6,Table 2.4 Construction Activities For Temporar...,0.75,894827.pdf,Application for the Edmonton to Hardisty Pipel...,Table


### Change this folder to the path were you want the tables saved

In [14]:
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\edmonton_hardisty')

### Change the dataframe name accordingly

In [15]:
files = list(edmonton_hardisty['DataID_pdf'].unique())
files

['894827.pdf',
 '894925.pdf',
 '894928.pdf',
 '895003.pdf',
 '895006.pdf',
 '895018.pdf',
 '895021.pdf',
 '895024.pdf',
 '895073.pdf',
 '895076.pdf',
 '895079.pdf',
 '895082.pdf',
 '895182.pdf',
 '895185.pdf',
 '895188.pdf',
 '895194.pdf',
 '895237.pdf',
 '895339.pdf']

### call the main function -- pass filename and dataframe as function arguments

In [ ]:
for file in files:
    extract_tables(file, edmonton_hardisty)

2020-01-29 14:08:12,430 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.23/tika-server-1.23.jar to C:\Users\T1MARG~1\AppData\Local\Temp\3\tika-server.jar.
2020-01-29 14:08:45,749 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.23/tika-server-1.23.jar.md5 to C:\Users\T1MARG~1\AppData\Local\Temp\3\tika-server.jar.md5.
2020-01-29 14:08:46,987 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


[]
No table on page 1 is detected
['Table 2.1 Technical Details – Edmonton To Hardisty Pipeline Project']
[]
No table on page 3 is detected
[]
No table on page 4 is detected
['Table 2.2 Pipeline Construction Activities']
['Table 2.2 Pipeline Construction Activities', 'Table 2.3 Construction Activities For Pump Station_Terminal Facilities', 'Table 2.4 Construction Activities For Temporary Facilities']
['Table 2.4 Construction Activities For Temporary Facilities', 'Table 2.5 Environmental Permits_Approvals']
['Table 2.6 Other Potential Environmental Permits_Approvals']
['Table 2.7 Expected Duration Of Major Construction And Operation Activities', 'Table 2.8 Operation And Maintenance Activities']
['Table 2.8 Operation And Maintenance Activities']
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
['Table 3.1 Consultation Activities With Federal And Provincial Authorities']
['Table 3.1 Consultation Act

['Table 5.4 Historical Mean Monthly Streamflow (M3_S)  Summary For The Battle River (Station No. 05Fc008)', 'Table 5.5 Historical Mean Monthly Streamflow (M3_S)  Summary For Iron Creek (Station No. 05Fb002)']
[]
No table on page 41 is detected
[]
No table on page 42 is detected
[]
No table on page 43 is detected
['Table 5.6 2011 Average Annual Ambient Air Quality Data In The Air Quality Rsa', 'Table 5.7 Reported Emissions (In Tonnes) From Facilities In The   Air Quality Rsa Of The Pipeline Route (2010)']
['Table 5.7 Reported Emissions (In Tonnes) From Facilities In The   Air Quality Rsa Of The Pipeline Route (2010)']
[]
No table on page 46 is detected
[]
No table on page 47 is detected
['Table 5.8 Summary Of Proposed Watercourse And Fish-Bearing Wetland Crossings Along The Proposed Pipeline Route']
['Table 5.9 Fish Species That May Occur In The North Saskatchewan River Near The  Proposed Pipeline Route']
['Table 5.10 Fish Species That May Occur In The Battle River  Near The Proposed Pi

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
Page 7 contains an image
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
['Table 1 Wetland Classes Established By Stewart And Kantrud']
['Table 1 Wetland Classes Established By Stewart And Kantrud', 'Table 2 Wetland Classes Established By The Canadian Wetland Classification System']
['Table 2 Wetland Classes Established By The Canadian Wetland Classification System']
[]
No table on page 17 is detected
['Table 3 Potential Wetlands Identified Within Additional Temporary Workspace']
['Table 3 Potential Wetlands Identified Within Additional Temporary Workspace']
[]
Page 20 contains an image
[]
No table on page 21 is detected
['Ta

...

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
['Table 1.  Basic Night-Time Sound Levels (As Per Ercb Directive 038)']
['Table 2.  Overall Leq Sound Levels For Kingman Station']
['Table 3.  Equipment Operating Conditions For Kingman Station']
['Table 4.  Calibration Case Modeled Results At Noise Monitor Location', 'Table 5.  Baseline Case Modeled Sound Levels']
['Table 6.  Baseline Case Modeled Dba And Dbc Sound Levels']
['Table 7.  Cumulative Case Modeled Sound Levels']
['Table 8.  Cumulative Case Modeled Dba And Dbc Sound Levels']
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
['Table 1.  Basic Night-Time Sound Levels (As Per Ercb Directive 038)']
['Table 2.  Overall Leq Sound Levels For Strome Station1']
['Table 3.  Equipment Operating Conditions For Strome Station']
[]
['Table 6.  Baseline Case Modeled Dba And Dbc Sound Levels']
['Table 7.  Cumulative Case Modeled Sound Levels']
['Table 8.  Cumulative Case Modeled Dba And Dbc Sound Levels']
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
[]
[]
No table on page 26 is detected
[]
[]
[]
Page 29 contains an image
[]
[]
[]
[]
[]
[]
[]
No table o

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected


## Application for Line 9 Reversal Phase I Project

In [13]:
hearing = 'Application for Line 9 Reversal Phase I Project'
line9 = df[df['Application title short'] == hearing].reset_index(drop = True)
line9.head()

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,13,Table 1-1 Federal Permits And Authorizations R...,1.0,705999.pdf,Application for Line 9 Reversal Phase I Project,Table
1,14,Table 1-2 Provincial Permits And Authorization...,1.0,705999.pdf,Application for Line 9 Reversal Phase I Project,Table
2,17,Table 2-1 Location Of Project Sites – Line 9 R...,1.0,705999.pdf,Application for Line 9 Reversal Phase I Project,Table
3,18,Table 2-2 Enbridge’S Construction Manual Prote...,1.0,705999.pdf,Application for Line 9 Reversal Phase I Project,Table
4,19,Table 3-1 Summary Of Regulatory Consultation A...,1.0,705999.pdf,Application for Line 9 Reversal Phase I Project,Table


### Change this folder to the path were you want the tables saved

In [14]:
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\line9')

### Change the dataframe name accordingly

In [16]:
files = list(line9['DataID_pdf'].unique())
for file in files:
    extract_tables(file, line9)

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
Page 12 contains an image
['Table 1-1 Federal Permits And Authorizations Required For The Modification Sites']
['Table 1-2 Provincial Permits And Authorizations Required For The Line 9 Modifications']
[]
No table on page 15 is detected
[]
No table on page 16 is detected
['Table 2-1 Location Of Project Sites – Line 9 Reversal Phase I Between Sarnia And North']
['Table 2-2 Enbridge’S Construction Manual Protection Measures']
['Table 3-1 Summary Of Regulatory Consultation Activities', 'Table 3-2 Summary Of Main Issues Raised By Agencies During Consultation']
[]
No table on page 20 is detected


[]
No table on page 21 is detected


[]
No table on page 22 is detected
[]
[]
No table on page 24 is detected
[]
No table on page 25 is detected
[]
No table on page 26 is detected
[]
No table on page 27 is detected
[]
No table on page 28 is detected
[]
No table on page 29 is detected
[]
No table on page 30 is detected
['Table 4-1 Sar Species Known To Occur In The Laa Of The Modification Sites']
['Table 4-1 Sar Species Known To Occur In The Laa Of The Modification Sites']
['Table 4-1 Sar Species Known To Occur In The Laa Of The Modification Sites']
['Table 4-2 General Soil Characteristics At The Modification Sites']
['Table 4-2 General Soil Characteristics At The Modification Sites']
[]
No table on page 36 is detected
['Table 4-3. 4.3.4 Westover Terminal', 'Table 4-3. ']
No table on page 37 is detected
['Table 4-3 Summary Of Site Specific Hydrogeological Characteristics']
['Table 4-4 Land Use Characteristics At The Modification Sites']
['Table 4-4 Land Use Characteristics At The Modification Sites', 'Table 4-5 Op  Designat

[]

## Application for the Leismer to Kettle River Crossover

In [17]:
hearing = 'Application for the Leismer to Kettle River Crossover'
leismer2kettle = df[df['Application title short'] == hearing].reset_index(drop = True)
leismer2kettle.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\leismer2kettle')
# Change the dataframe name accordingly
files = list(leismer2kettle['DataID_pdf'].unique())
for file in files:
    extract_tables(file, leismer2kettle)

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,25,Table 6-39: Predicted Residual Project Effects...,1.0,702727.pdf,Application for the Leismer to Kettle River Cr...,Table
1,36,Table 1-1: Summary Of Potential Environmental ...,1.0,702727.pdf,Application for the Leismer to Kettle River Cr...,Table
2,38,Table 1-2: Leismer To Kettle River Crossover P...,1.0,702727.pdf,Application for the Leismer to Kettle River Cr...,Table
3,39,Table 1-3: Concordance With Guide A – A.2 Envi...,1.0,702727.pdf,Application for the Leismer to Kettle River Cr...,Table
4,40,Table A-3 Circumstances Triggering The Need Fo...,1.0,702727.pdf,Application for the Leismer to Kettle River Cr...,Table


[]
Page 1 contains an image
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
No table on page 24 is detected
['Table 6-39  Predicted Residual Project Effects Description And Importance Of Effects To Social And Cultural Well-Being .. 6-114']
No table on page 25 is detected
[]
No table

[]
No table on page 162 is detected
[]
No table on page 163 is detected
[]
No table on page 164 is detected
[]
Page 165 contains an image
[]
Page 166 contains an image
[]
Page 167 contains an image
[]
Page 168 contains an image
[]
No table on page 169 is detected
[]
No table on page 170 is detected
[]
No table on page 171 is detected
[]
No table on page 172 is detected
[]
No table on page 173 is detected
[]
No table on page 174 is detected
[]
No table on page 175 is detected
[]
No table on page 176 is detected
[]
No table on page 177 is detected
[]
No table on page 178 is detected
[]
No table on page 179 is detected
['Table 4-39  2009 Average Annual Daily Traffic (Aadt) On The Socio-Economic Study Area Highways']
[]
No table on page 181 is detected
[]
No table on page 182 is detected
[]
No table on page 183 is detected
['Table 4-40  Labour Force Activity In 2006 In The Socio-Economic Study Area']
['Table 4-41  Labour Force Activity In 2006 In The Aboriginal Population In The Socio-Econ

[]

[]
No table on page 1 is detected
[]
Page 2 contains an image
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
Page 10 contains an image
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
No table on page 24 is detected
['Table 1  Resource-Specific Mitigation Table']
[]
[]
No table on page 27 is detected
[]
No table on page 28 is detected
[]
No table on page 29 is detected
[]
No table on page 30 is

...

[]
Page 1 contains an image
[]
Page 2 contains an image
[]
Page 3 contains an image
[]
Page 4 contains an image
[]
Page 5 contains an image
[]
Page 6 contains an image
[]
Page 7 contains an image
[]
Page 8 contains an image
[]
Page 9 contains an image
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected
[]
No table on page 22 is detected
['Table D-1  Water Well Database Results']
[]
[]
Duration: 0:01:40.431153


[]

[]
No table on page 1 is detected
[]
Page 2 contains an image
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
Page 5 contains an image
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
Page 11 contains an image
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
Page 18 contains an image
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
Page 21 contains an image
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
No table on page 24 is detected
[]
No table on page 25 is detected
[]
No table on page 26 is detected
[]
No table on page 27 is detected
[]
No table on page 28 is detected
[]
Page 29 contains an image
[]
No table on page 30 is dete

[]
No table on page 76 is detected
[]
No table on page 77 is detected
Duration: 0:05:37.200233


[]

## Application for the Groundbirch Pipeline Project

In [18]:
hearing = 'Application for the Groundbirch Pipeline Project'
groundbirch = df[df['Application title short'] == hearing].reset_index(drop = True)
groundbirch.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\groundbirch')
# Change the dataframe name accordingly
files = list(groundbirch['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, groundbirch)

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,24,Table 1.1 Concordance With Guide A – A.2 Envir...,0.75,555069.pdf,Application for the Groundbirch Pipeline Project,Table
1,25,Table 1.1 Concordance With Guide A – A.2 Envir...,0.35,555069.pdf,Application for the Groundbirch Pipeline Project,Table
2,26,Table 1.1 Concordance With Guide A – A.2 Envir...,0.35,555069.pdf,Application for the Groundbirch Pipeline Project,Table
3,26,Table A-3 Circumstances Triggering The Need F...,1.00,555069.pdf,Application for the Groundbirch Pipeline Project,Table
4,27,Table A-3 Circumstances Triggering The Need F...,0.35,555069.pdf,Application for the Groundbirch Pipeline Project,Table


[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
Page 20 contains an image
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 is detected
['Table 1.1 Concordance With Guide A – A.2 Environment And Socio-Economic Assessment Of  The Neb Filing Manual']
['Table 1.1 Concordance With Guide A – A.2 Environment And Socio-Economic Assessment Of  The Neb 

[]

[]
No table on page 1 is detected
['Table 6.1 Evaluation Of The Significance Of Residual Effects -  Environmental And Socio-Economic Assessment Criteria1']
['Table 6.1 Evaluation Of The Significance Of Residual Effects -  Environmental And Socio-Economic Assessment Criteria1']
[]
Page 4 contains an image
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
No table on page 24 is detected

['Table 6.4 Potential Biophysical And Socio-Economic Effects, Mitigative Measures  And Residual Effects Of Meter Station Installation And Operation']
['Table 6.4 Potential Biophysical And Socio-Economic Effects, Mitigative Measures  And Residual Effects Of Meter Station Installation And Operation']
['Table 6.4 Potential Biophysical And Socio-Economic Effects, Mitigative Measures  And Residual Effects Of Meter Station Installation And Operation']
['Table 6.5 Significance Evaluation Of Adverse Residual Effects Of Identified Biophysical  And Socio-Economic Elements Of Meter Station Installation']
['Table 6.5 Significance Evaluation Of Adverse Residual Effects Of Identified Biophysical  And Socio-Economic Elements Of Meter Station Installation']
['Table 6.6 Elements Not Interacting With The Meter Station Installation Component Of  The Groundbirch Pipeline Project']
Duration: 0:04:20.340035


...

[]
No table on page 96 is detected


[]
No table on page 97 is detected


[]
No table on page 98 is detected


[]
No table on page 99 is detected
[]
No table on page 100 is detected
Duration: 0:04:57.953549


[]

In [19]:
hearing = 'Application for the Enbridge Northern Gateway Pipeline Project'
gateway = df[df['Application title short'] == hearing].reset_index(drop = True)
gateway.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\gateway')
# Change the dataframe name accordingly
files = list(gateway['DataID_pdf'].unique())
for file in files:
    extract_tables(file, gateway)

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,15,Table 10-1 Potential Environmental Effects On ...,1.0,619816.pdf,Application for the Enbridge Northern Gateway ...,Table
1,16,Table 10-1 Potential Environmental Effects On ...,1.0,619816.pdf,Application for the Enbridge Northern Gateway ...,Table
2,17,Table 10-1 Potential Environmental Effects On ...,1.0,619816.pdf,Application for the Enbridge Northern Gateway ...,Table
3,31,Table 10-2 Characterization Of The Residual Ef...,1.0,619816.pdf,Application for the Enbridge Northern Gateway ...,Table
4,32,Table 10-2 Characterization Of The Residual Ef...,1.0,619816.pdf,Application for the Enbridge Northern Gateway ...,Table


[]
No table on page 1 is detected
[]
Page 2 contains an image
[]
Page 3 contains an image
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
['Table 10-1 Potential Environmental Effects On Hydrology This Table Identifies The Potential Environmental Effects On Hydrology That Are Assessed In This Section Of The Esa. Each Of These Environmental Effects Is [...]']
file 619816.pdf failed to open
Duration: 0:01:27.713874


['619816.pdf']

[]
No table on page 1 is detected
[]
No table on page 2 is detected
['Table 11-1 Mapped Watercourse Crossings By Major Watershed In Alberta And British Columbia']
[]
Page 4 contains an image
[]
[]
[]
[]
No table on page 8 is detected
[]
No table on page 9 is detected
['Table 11-2 Summary Of Key Issues For Fish And Fish Habitat']
['Table 11-2 Summary Of Key Issues For Fish And Fish Habitat (Cont’D)']
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
['Table 11-3 Recommended Watercourse Crossing Changes  Recommended Watercourse Crossing Location Change']
[]
Page 20 contains an image
[]
No table on page 21 is detected
[]
Page 22 contains an image
[]
No table on page 23 is detected
[]
No table on page 24 is detected
[]
No table on page 25 is detected
[]
No table on page 26 is detected
['Table 11-4 

[]
Page 70 contains an image
[]
No table on page 71 is detected
[]
No table on page 72 is detected
[]
No table on page 73 is detected
[]
No table on page 74 is detected
[]
No table on page 75 is detected
['Table 11-18 Mitigation For Potential Effects On Freshwater Fish Habitat Productive Capacity']
['Table 11-18 Mitigation For Potential Effects On Freshwater Fish Habitat Productive Capacity (Cont’D)']
['Table 11-18 Mitigation For Potential Effects On Freshwater Fish Habitat Productive Capacity (Cont’D)']
[]
No table on page 79 is detected
[]
No table on page 80 is detected
[]
No table on page 81 is detected
[]
No table on page 82 is detected
[]
['Table 11-19 Risk Determination Results']


...

[]

[]
No table on page 1 is detected
[]
Page 2 contains an image
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
['Table 11-1 Potential Environmental Effects On Marbled Murrelet This Table Identifies The Potential Environmental Effects On Marbled Murrelet That Are Assessed In This Section Of The Esa. Each Of These [...]']
file 620256.pdf failed to open
Duration: 0:01:13.921098


['620256.pdf']

[]
No table on page 1 is detected
['Table 13-8 Fishery Openings And Closures In The Open Water Area  Month']
['Table 13-8 Fishery Openings And Closures In The Open Water Area (Cont’D)  Month']
['Table 13-9 Overlap Of Approaches And Commercial Fishing Efforts In The Open Water Area']
[]
Page 5 contains an image
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
Page 8 contains an image
[]
Page 9 contains an image
[]
No table on page 10 is detected
[]
Page 11 contains an image
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
No table on page 24 is detected
[]
Page 25 contains an image
[]
No table on p

[]

In [20]:
hearing = 'Application for the Horn River Project'
horn_river = df[df['Application title short'] == hearing].reset_index(drop = True)
horn_river.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\horn_river')
# Change the dataframe name accordingly
files = list(horn_river['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, horn_river)

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,9,Table 1 Locations Of Supplemental Summer Wildl...,0.75,631608.pdf,Application for the Horn River Project,Table
1,15,Table 2 Wetlands Containing Amphibians Along T...,1.00,631608.pdf,Application for the Horn River Project,Table
2,4,Table 1 Ungulate Winter Ranges And Wildlife Ha...,0.75,631617.pdf,Application for the Horn River Project,Table
3,5,Table 2 Summary Of Project Consultation Relate...,0.75,631617.pdf,Application for the Horn River Project,Table
4,11,Table 3 Project Components And Rare Plant Surv...,0.75,631734.pdf,Application for the Horn River Project,Table


['631608.pdf', '631617.pdf', '631734.pdf', '631814.pdf', '631817.pdf', '631826.pdf', '631829.pdf']
[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
Page 6 contains an image
[]
No table on page 7 is detected
[]
No table on page 8 is detected
['Table 1 Locations Of Supplemental Summer Wildlife Surveys Along The  Horn River Mainline Project - Summer 20101']
[]
No table on page 10 is detected
[]
Page 11 contains an image
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
['Table 2 Wetlands Containing Amphibians Along The Horn River Mainline Project In Summer 2010']
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
[]
Page 22 contains an image
[]
[]
[]
[]
Page 26 contains an image
[]
[]
[]


[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
['Table 1 Ungulate Winter Ranges And Wildlife Habitat Area1']
['Table 2 Summary Of Project Consultation Related To Woodland Caribou']
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
Duration: 0:00:16.932437


[]

...

[]

[]
No table on page 1 is detected
['Table 1 2009 And 2010 Rare Plant Survey Locations']
['Table 1 2009 And 2010 Rare Plant Survey Locations']
[]
Page 4 contains an image
['Table 2 Summary Of Rare Plant Species Observations  Along Or Near The Proposed Horn River Mainline Project']
[]
Page 6 contains an image
[]
[]
[]
Page 9 contains an image
[]
[]
Page 11 contains an image
[]
Page 12 contains an image
Duration: 0:02:39.206184


[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
Page 9 contains an image
[]
No table on page 10 is detected
['Table 1 Status Of Traditional Land Use Studies']
[]
No table on page 12 is detected
[]
No table on page 13 is detected
Duration: 0:00:22.562593


[]

In [21]:
hearing = 'Application for Bakken Pipeline Project Canada'
bakken = df[df['Application title short'] == hearing].reset_index(drop = True)
bakken.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\bakken')
# Change the dataframe name accordingly
files = list(bakken['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, bakken)

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,6,Table 1.6‐1. Concordance With The Guide A – A...,0.75,661244.pdf,Application for Bakken Pipeline Project Canada,Table
1,8,Table A‐3 Circumstances Triggering The Need Fo...,0.75,661244.pdf,Application for Bakken Pipeline Project Canada,Table
2,9,Table 2.1‐1. Physical Components Required For...,0.75,661244.pdf,Application for Bakken Pipeline Project Canada,Table
3,10,Table 2.1‐2. Proposed Project Coordinates 1 C...,0.75,661244.pdf,Application for Bakken Pipeline Project Canada,Table
4,11,Table 2.4‐1. Summary Of Construction Activiti...,0.75,661244.pdf,Application for Bakken Pipeline Project Canada,Table


['661244.pdf', '661247.pdf', '661253.pdf', '661256.pdf', '661447.pdf', '661450.pdf', '661456.pdf', '661459.pdf', '661462.pdf', '661465.pdf', '661468.pdf', '661471.pdf', '661496.pdf', '661509.pdf', '661512.pdf', '661515.pdf', '661574.pdf', '661577.pdf', '661587.pdf', '661611.pdf', '661705.pdf', '661708.pdf', '661714.pdf', '661717.pdf', '661720.pdf', '661723.pdf', '661726.pdf']
[]
[]
Page 2 contains an image
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
['Table\xa01.6‐1.\xa0 Concordance\xa0With\xa0The\xa0Guide\xa0A\xa0–\xa0A.2\xa0Environmental\xa0And\xa0Socio‐Economic\xa0Assessment\xa05\xa0Of\xa0The\xa0Neb\xa0Filing\xa0Manual\xa06 Filing\xa0#\xa0 Filing\xa0Requirement\xa0 Esa\xa0Section']
[]
['Table\xa0A‐3\xa0Circumstances\xa0Triggering\xa0The\xa0Need\xa0For\xa0Detailed\xa0Biophysical\xa0And\xa0Socio‐Economic\xa0Information Physical\xa0Environment\xa0\xa0 Section\xa05.0']
['Table\xa02.1‐1.\xa0 Physical\xa0Components\xa0Required\xa0F

[]

[]
Page 1 contains an image
[]
No table on page 2 is detected
[]
No table on page 3 is detected
['Table\xa05.1‐4.\xa0 Summary\xa0Of\xa0Valued\xa0Components*\xa01 Valued\xa0Component\xa0 Vc\xa0Identification']
file 661247.pdf failed to open
Duration: 0:03:21.471261


['661247.pdf']

...

[]

[]
No table on page 1 is detected
['Table\xa0K‐1\xa0 Species\xa0Status\xa0According\xa0To\xa0The\xa0Saskatchewan\xa0Conservation\xa0Data\xa0Centre\xa0(Sk\xa0Cdc\xa02010) Rank\xa0 Status\xa0 Definition\xa0 Comments', 'Table\xa0K‐2\xa0 Species\xa0Status\xa0According\xa0To\xa0The\xa0Manitoba\xa0Endangered\xa0Species\xa0Act\xa0(2010) Status\xa0 Definition', 'Table\xa0K‐3\xa0 Species\xa0Status\xa0According\xa0To\xa0Canadian\xa0Species\xa0At\xa0Risk\xa0(Cosewic\xa02010) Status\xa0 Definition']
['Table\xa0K‐4\xa0 Species\xa0Status\xa0According\xa0To\xa0The\xa0Species\xa0At\xa0Risk\xa0Act\xa0(Sara\xa02010) Status\xa0 Definition']
['Table\xa0K‐5\xa0 Saskatchewan\xa0Conservation\xa0Data\xa0Centre\xa0Ranking\xa0System\xa0For\xa0Rare\xa0Plants Standard\xa0Subnational\xa0Conservation\xa0Status\xa0Ranks']
['Table\xa0K‐6\xa0 Federal\xa0Species\xa0At\xa0Risk\xa0Act\xa0(Sara)\xa0Rare\xa0Plant\xa0Species\xa0Conservation\xa0Ranks Federal\xa0Sara\xa0Rank\xa0 Rank\xa0Description']
Duration: 0:00:10.002262


[]

[]
No table on page 1 is detected
['Table Of Contents M1.0 Introduction ............................................................................................................................ 775 M2.0 The Study Area [...]', 'Tables Table\xa0M‐1.\xa0 Areal\xa0Extent\xa0Of\xa0The\xa0Soil\xa0Subgroup\xa0At\xa0The\xa0Bakken\xa0Pump\xa0Station\xa0And\xa0Access\xa0Road\xa0 Area', 'Table\xa0M‐1.\xa0 Areal\xa0Extent\xa0Of\xa0The\xa0Soil\xa0Subgroup\xa0At\xa0The\xa0Bakken\xa0Pump\xa0Station\xa0And\xa0Access\xa0Road\xa0Area Table\xa0M‐2.\xa0 Soil\xa0Inspection\xa0Inventory\xa0Of\xa0The\xa0Soils\xa0Found\xa0At\xa0The\xa0Bakken\xa0Pump\xa0Station\xa0And\xa0 Access\xa0Road', 'Table M‐2. Soil Inspection Inventory Of The Soils Found At The Bakken Pump Station And Access Road Table M‐3. Extent Of Soil Phases At The Bakken Pump Station And Access Road Table M‐4. Soil Quality Results – [...]', 'Table M‐3. Extent Of Soil Phases At The Bakken Pump Station And Access Road Table M‐4. Soil Quality Resul

[]

In [22]:
hearing = 'Application for the Vantage Pipeline Project'
vantage = df[df['Application title short'] == hearing].reset_index(drop = True)
vantage.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\vantage')
# Change the dataframe name accordingly
files = list(vantage['DataID_pdf'].unique())
for file in files:
    extract_tables(file, vantage)    

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,2,"Table 10-1 Legislation, Guidelines And Policie...",0.75,667026.pdf,Application for the Vantage Pipeline Project,Table
1,4,Table 10-2 Interaction Of The Project With Gro...,0.75,667029.pdf,Application for the Vantage Pipeline Project,Table
2,6,Table 10-3 Evaluation Of The Significance Of R...,0.75,667029.pdf,Application for the Vantage Pipeline Project,Table
3,8,Table 10-4 Analysis Of Potential Residual Effe...,0.75,667029.pdf,Application for the Vantage Pipeline Project,Table
4,10,Table 10-5 Analysis Of Potential Cumulative Re...,0.75,667029.pdf,Application for the Vantage Pipeline Project,Table


[]
No table on page 1 is detected
['Table 10-1 Legislation, Guidelines And Policies Related To Water Quality And Quantity']
[]
No table on page 3 is detected
[]
No table on page 4 is detected
Duration: 0:00:07.378193


[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
['Table 10-2 Interaction Of The Project With Groundwater Quantity And Quality']
[]
['Table 10-3 Evaluation Of The Significance Of Residual Effects']
[]
['Table 10-4 Analysis Of Potential Residual Effects On Groundwater Quantity And Quality']
[]
No table on page 9 is detected
['Table 10-5 Analysis Of Potential Cumulative Residual Effects On Groundwater Quantity And Quality']
[]
No table on page 11 is detected
[]
No table on page 12 is detected
Duration: 0:00:34.415908


[]

...

[]
No table on page 3 is detected
['Table 15-4 First Nations Within The Vantage Pipeline Rsa']


[]
No table on page 5 is detected
[]
No table on page 6 is detected
Duration: 0:00:10.786287


[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
['Table 19-1 Project Inclusion List']
[]
[]
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
['Table 19-2 Vantage 2010 Cumulative Effects Summary Table']
[]
[]
[]
[]
[]
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
No table on page 24 is detected
Duration: 0:01:16.675026


[]

In [23]:
hearing = 'Application for the Northwest Mainline Expansion'
northwest_mainline = df[df['Application title short'] == hearing].reset_index(drop = True)
northwest_mainline.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\northwest_mainline')
# Change the dataframe name accordingly
files = list(northwest_mainline['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, northwest_mainline)  

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,2,Table 6.1 Environmental And Socio-Economic Eff...,0.75,684259.pdf,Application for the Northwest Mainline Expansion,Table
1,3,Table 6.2 Evaluation Of The Significance Of Re...,0.75,684259.pdf,Application for the Northwest Mainline Expansion,Table
2,4,Table 6.2 Evaluation Of The Significance Of Re...,0.35,684259.pdf,Application for the Northwest Mainline Expansion,Table
3,8,Table 6.3 Element Interaction With Proposed Pi...,0.75,684259.pdf,Application for the Northwest Mainline Expansion,Table
4,10,"Table 6.4 Potential Effects, Mitigative Measur...",0.75,684259.pdf,Application for the Northwest Mainline Expansion,Table


['684259.pdf', '684377.pdf', '684380.pdf', '684383.pdf', '684386.pdf', '684447.pdf', '684450.pdf', '684491.pdf', '684494.pdf', '684512.pdf', '684515.pdf']
[]
No table on page 1 is detected
['Table 6.1 Environmental And Socio-Economic Effects Assessment Team']
['Table 6.2 Evaluation Of The Significance Of Residual Effects -  Environmental And Socio-Economic Assessment Criteria1']
['Table 6.2 Evaluation Of The Significance Of Residual Effects -  Environmental And Socio-Economic Assessment Criteria1']
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
['Table 6.3 Element Interaction With Proposed Pipeline Component']
[]
Page 9 contains an image
['Table 6.4 Potential Effects, Mitigative Measures And Residual Effects Of Pipeline  Construction And Operation On The Physical And Meteorological Environment']
['Table 6.4 Potential Effects, Mitigative Measures And Residual Effects Of Pipeline  Construction And Operation On The Physical And Meteor

['Table 6.23 Significance Evaluation Of Potential Residual Effects Of Pipeline   Construction And Operation On Vegetation']
['Table 6.23 Significance Evaluation Of Potential Residual Effects Of Pipeline   Construction And Operation On Vegetation']
[]
No table on page 98 is detected
[]
No table on page 99 is detected
[]
No table on page 100 is detected
[]
No table on page 101 is detected
Duration: 0:08:52.173561


[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
No table on page 24 is detected
[]
No table on page 25 is detected
[]
No table on page 26 is detected
[]
No table on page 27 is detected
[]
No table on page 28 is detected
[]
No table on page 29 is det

...

[]
No table on page 100 is detected


[]
No table on page 101 is detected


[]
No table on page 102 is detected


[]
No table on page 103 is detected
[]
No table on page 104 is detected
[]
No table on page 105 is detected
[]
No table on page 106 is detected
[]
No table on page 107 is detected
[]
No table on page 108 is detected
[]
No table on page 109 is detected
[]
No table on page 110 is detected
[]
[]
No table on page 112 is detected
[]
No table on page 113 is detected
[]
No table on page 114 is detected
[]
No table on page 115 is detected
[]
No table on page 116 is detected
[]
No table on page 117 is detected
[]
No table on page 118 is detected
[]
No table on page 119 is detected
[]
No table on page 120 is detected
[]
No table on page 121 is detected
[]
No table on page 122 is detected
[]
No table on page 123 is detected
[]
No table on page 124 is detected
[]
Page 125 contains an image
[]
No table on page 126 is detected
[]
No table on page 127 is detected
[]
No table on page 128 is detected
[]
No table on page 129 is detected
[]
No table on page 130 is detected
[]
No table on page 131 is dete

[]

In [24]:
hearing = 'Application for the Line 4 Extension Project'
line4 = df[df['Application title short'] == hearing].reset_index(drop = True)
line4.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\line4')
# Change the dataframe name accordingly
files = list(line4['DataID_pdf'].unique())
for file in files:
    extract_tables(file, line4) 

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,35,Table 6.1 Evaluation Of The Significance Of Re...,0.75,467846.pdf,Application for the Line 4 Extension Project,Table
1,36,Table 6.2 Potential Biophysical And Socio-Econ...,0.75,467846.pdf,Application for the Line 4 Extension Project,Table
2,37,Table 6.2 Potential Biophysical And Socio-Econ...,0.35,467846.pdf,Application for the Line 4 Extension Project,Table
3,38,Table 6.2 Potential Biophysical And Socio-Econ...,0.35,467846.pdf,Application for the Line 4 Extension Project,Table
4,39,Table 6.2 Potential Biophysical And Socio-Econ...,0.35,467846.pdf,Application for the Line 4 Extension Project,Table


[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
No table on page 24 is detected
[]
No table on page 25 is detected
[]
No table on page 26 is detected
[]
No table on page 27 is detected
[]
No table on page 28 is detected
[]
No table on page 29 is det

[]
No table on page 79 is detected


[]
No table on page 80 is detected
[]
No table on page 81 is detected
[]
No table on page 82 is detected
[]
No table on page 83 is detected
[]
No table on page 84 is detected
[]
[]
[]
Page 87 contains an image
[]
Page 88 contains an image
[]
[]
[]
Page 91 contains an image
[]
Page 92 contains an image
[]
Page 93 contains an image
[]
Page 94 contains an image
[]
Page 95 contains an image
[]
Page 96 contains an image
[]
Page 97 contains an image
[]
Page 98 contains an image
[]
Page 99 contains an image
[]
Page 100 contains an image
[]
Page 101 contains an image
[]
Page 102 contains an image
[]
[]
Page 104 contains an image
[]
Page 105 contains an image
[]
Page 106 contains an image
[]
Page 107 contains an image
[]
[]
Page 109 contains an image
[]
Page 110 contains an image
[]
Page 111 contains an image
[]
Page 112 contains an image
[]
Page 113 contains an image
[]
Page 114 contains an image
[]
Page 115 contains an image
[]
Page 116 contains an image
[]
Page 117 contains an image
[]
[]
Pa

[]

...

[]
No table on page 1 is detected
['Table 5.1 Physiographic Divisions Traversed By The Proposed  Line 4 Extension Pipeline Route']
[]
Page 3 contains an image
['Table 5.2 Surficial Geology Traversed By The Proposed Line 4 Extension Pipeline Route']
['Table 5.3 Snowfall, Rainfall And Temperature Averages (1971-2000)']
['Table 5.4 Product Releases In The Vicinity Of The Proposed Pipeline Route']
['Table 5.4 Product Releases In The Vicinity Of The Proposed Pipeline Route']
['Table 5.4 Product Releases In The Vicinity Of The Proposed Pipeline Route']
[]
No table on page 9 is detected
[]
No table on page 10 is detected
['Table 5.5 Summary Of Soil Characteristics Along The Edmonton Loop']
['Table 5.5 Summary Of Soil Characteristics Along The Edmonton Loop']
['Table 5.6 Summary Of Soil Characteristics Along The Kingman Loop']
['Table 5.6 Summary Of Soil Characteristics Along The Kingman Loop']
['Table 5.7 Summary Of Soil Characteristics Along The Strome Loop']
['Table 5.7 Summary Of Soil Char

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
['Table 1.  Total Estimated Indirect Ghg Emissions Contribution Due To The Line 4 Extension Project.']
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
Page 10 contains an image
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
['Table 1 Wetland Types Encounted By The Proposed Pipeline Route', 'Table 2 Wetland Classes Established By Stewart And Kantrud (1971)']
['Table 2 Wetland Classes Established By Stewart And Kantrud (1971)']
[]
No table on page 16 is detected
[]
No table on page 17 is detected
['Table 3 Summary Of Wetlands Found Along Total Length']
['Table 4 Wetland Distribution And Description Along The']
['Table 4 Wetland Distribution And Description Along The', 'Table 5 Wetland Distribution And Descri

[]
Page 55 contains an image
[]
Page 56 contains an image
[]
Page 57 contains an image
Duration: 0:04:24.774049


[]

In [26]:
hearing = 'Application for the Cushing Expansion'
cushing_expansion = df[df['Application title short'] == hearing].reset_index(drop = True)
cushing_expansion.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\cushing_expansion')
# Change the dataframe name accordingly
files = list(cushing_expansion['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, cushing_expansion) 

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,32,Table A-1 Summary Of Erosion And Sediment Cont...,1.0,486224.pdf,Application for the Cushing Expansion,Table
1,33,Table A-1 Summary Of Erosion And Sediment Cont...,1.0,486224.pdf,Application for the Cushing Expansion,Table
2,34,Table A-1 Summary Of Erosion And Sediment Cont...,1.0,486224.pdf,Application for the Cushing Expansion,Table
3,2,Table 10.1 Physical Works And Activities Consi...,1.0,486452.pdf,Application for the Cushing Expansion,Table
4,18,Table 1-1 Physical Works And Activities Consid...,1.0,486461.pdf,Application for the Cushing Expansion,Table


['486224.pdf', '486452.pdf', '486461.pdf', '486464.pdf']
[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
No table on page 24 is detected
[]
No table on page 25 is detected
[]
No table on page 26 is detected
[]
No table on page 27 is detected
[]
No t

[]

[]
No table on page 1 is detected
['Table 10.1 Physical Works And Activities Considered In The Esa']
[]
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
[]
[]
[]
[]
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
Duration: 0:00:29.354290


[]

...

['486461.pdf']

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
Page 3 contains an image
[]
No table on page 4 is detected


[]
No table on page 5 is detected
[]
No table on page 6 is detected


[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
Page 9 contains an image
['Table 10-1 Key Socio-Economic Issues  Project Activities And']
['Table 10-2 Capital Cost Estimate For Cushing Expansion Facilities Construction Component']
['Table 10-3 Electrical Power Requirements', 'Table 10-4 Access Road Characteristics For Proposed Pump Stations']
['Table 10-5 Population Of Regional Study Area']
['Table 10-6 Population Of Major Urban Centres Within The Rsa Of The Cushing Expansion Facilities']
['Table 10-7 Population Of Aboriginal Communities Within The Rsa First Nation Province Population (On Reserve)1']
['Table 10-8 Rural Municipalities Where The Proposed Pump Stations Will Be Located']
['Table 10-9 Summary Of Available Accommodation Units Within The Rsa Of The Proposed Pump Stations.']
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
Duration: 0:00:45.684915


[]

In [27]:
hearing = 'Application for Redwillow Pipeline Project'
red_willow = df[df['Application title short'] == hearing].reset_index(drop = True)
red_willow.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\red_willow')
# Change the dataframe name accordingly
files = list(red_willow['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, red_willow) 

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,25,Table 1.1 Land Disposition Holders Adjacent To...,0.75,487606.pdf,Application for Redwillow Pipeline Project,Table
1,29,Table 1.2 Concordance With Guide A – A.2 Envir...,0.75,487606.pdf,Application for Redwillow Pipeline Project,Table
2,31,Table A-3 Circumstances Triggering The Need Fo...,1.00,487606.pdf,Application for Redwillow Pipeline Project,Table
3,1,Table 2.1 Existing Facilities Located Within 5...,0.75,487609.pdf,Application for Redwillow Pipeline Project,Table
4,2,Table 2.1 Existing Facilities Located Within 5...,0.35,487609.pdf,Application for Redwillow Pipeline Project,Table


['487606.pdf', '487609.pdf', '487612.pdf', '487791.pdf', '487797.pdf', '487800.pdf', '487803.pdf', '487806.pdf', '487809.pdf', '487812.pdf', '487815.pdf', '487853.pdf', '487856.pdf', '487862.pdf', '487868.pdf', '487871.pdf', '487877.pdf', '487880.pdf', '487929.pdf', '487932.pdf', '487944.pdf', '487953.pdf', '487956.pdf', '487959.pdf', '487962.pdf', '487965.pdf']


[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
Page 23 contains an image
[]
No table on page 24 is detected
['Table 1.1 Land Disposition Holders Adjacent To The Pipeline Route']
[]
No table on page 26 is detected
[]
No table on page 27 is detected
[]
No table on page 28 is detected


[]

['Table 2.1 Existing Facilities Located Within 5 Km Of The Proposed  Redwillow Pipeline Project']
['Table 2.1 Existing Facilities Located Within 5 Km Of The Proposed  Redwillow Pipeline Project', 'Table 2.2 Mid-Case Estimates Of Gas-In-Place In British Columbia']
['Table 2.2 Mid-Case Estimates Of Gas-In-Place In British Columbia']
[]
No table on page 4 is detected
['Table 2.3 Technical Details Of The Project']
[]
No table on page 6 is detected
['Table 2.4 Details Of Permanent Pipeline Facilities']
[]
No table on page 8 is detected
['Table 2.5 Summary Of Environmental Field Surveys']
[]
No table on page 10 is detected
[]
No table on page 11 is detected
['Table 2.6 Environmental Permits Or Approvals Required  Prior To Construction Of The Redwillow Pipeline Project']
['Table 2.7 Additional Environmental Permits Potentially  Associated With The Redwillow Pipeline Project']
[]
Page 14 contains an image
['Table 2.8 Elements Of Pipeline System Commissioning']
['Table 2.8 Elements Of Pipeline 

...

[]

['Table 4 Summary Of Rare Plant Species Observed During The 2007 Surveys']
['Table 4  Con’T Common Name Scientific Name Provincial Rank Global Rank  Number Of', "Table 5 Bog Adder'S-Mouth Observations"]
['Table 6 Pinesap Observations']
['Table 6 Pinesap Observations']
["Table 7 Western Jacob'S-Ladder Observations", 'Table 8 Dainty Moonwort Observations']
['Table 9 Autumn Willow Observations']
['Table 10 Northern Bog Bedstraw Observations', 'Table 11 Small Bedstraw Observations']
['Table 11 Small Bedstraw Observations', 'Table 12 Golden Saxifrage Observations']
['Table 12 Golden Saxifrage Observations', 'Table 13 Green Saxifrage Observations']
['Table 13 Green Saxifrage Observations']
['Table 14 Yellow Monkeyflower Observations']
['Table 14 Yellow Monkeyflower Observations']
['Table 15 Conic Liverwort Observations', 'Table 16 Yellow Collar Moss Observations']
['Table 17 Red Collar Moss Observations']
['Table 18 Non-Native, Invasive Species That Occur Along The Proposed Route']
['Table 1

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
['Table 1 Areas Assessed Under Permit 07-368']
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
['Table 2 Results Of Historical Resources Impact Assessment']
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
Page 15 contains an image
[]
Page 16 contains an image
[]
Page 17 contains an image
[]
Page 18 contains an image
[]
Page 19 contains an image
[]
Page 20 contains an image
[]
No table on page 21 is detected
[]
[]
[]
[]
No table on page 25 is detected
[]
No table on page 26 is detected
[]
No table on page 27 is detected
[]
No table on page 28 is detected
[]
No table on page 29 is detected
[]
No table on page 30 is detected
Duration: 0:01:38.721606


[]

In [28]:
hearing = 'Application to construct and operate the South Peace Pipeline Project'
south_peace = df[df['Application title short'] == hearing].reset_index(drop = True)
south_peace.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\south_peace')
# Change the dataframe name accordingly
files = list(south_peace['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, south_peace) 

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,2,Table 5.1 Bedrock Geology Traversed By The Pro...,0.75,500409.pdf,Application to construct and operate the South...,Table
1,3,Table 5.2 Surficial Geology Traversed By The P...,0.75,500409.pdf,Application to construct and operate the South...,Table
2,4,Table 5.3 Earthquakes Felt In The Vicinity Of ...,0.75,500409.pdf,Application to construct and operate the South...,Table
3,5,Table 5.3 Earthquakes Felt In The Vicinity Of ...,0.35,500409.pdf,Application to construct and operate the South...,Table
4,5,"Table 5.4 Snowfall, Rainfall And Temperature A...",0.75,500409.pdf,Application to construct and operate the South...,Table


['500409.pdf', '500412.pdf', '500415.pdf', '500418.pdf', '500495.pdf', '500498.pdf', '500624.pdf', '500630.pdf', '500633.pdf', '500636.pdf', '500642.pdf']
[]
No table on page 1 is detected
['Table 5.1 Bedrock Geology Traversed By The Proposed Project']
['Table 5.2 Surficial Geology Traversed By The Project']
['Table 5.3 Earthquakes Felt In The Vicinity Of The Project Area']
['Table 5.3 Earthquakes Felt In The Vicinity Of The Project Area', 'Table 5.4 Snowfall, Rainfall And Temperature Averages (1971-2000)  For Dawson Creek, Bc']
[]
No table on page 6 is detected
['Table 5.5 Cli Soil Capability For Agriculture Along The Project']
['Table 5.5 Cli Soil Capability For Agriculture Along The Project']
['Table 5.6 Active Surface Water Licences In The Vicinity Of The Project']
['Table 5.6 Active Surface Water Licences In The Vicinity Of The Project']
[]
No table on page 11 is detected
['Table 5.7 Water Wells Located In The Vicinity Of The Project']
['Table 5.8 Reported Emissions (In Tonnes) Fr

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
[]
No table on page 4 is detected
['Table 6.1 Evaluation Of The Significance Of Residual Effects -  Environmental And Socio-Economic Assessment Criteria1']
['Table 6.1 Evaluation Of The Significance Of Residual Effects -  Environmental And Socio-Economic Assessment Criteria1']
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
No table on page 24 is detected
[]
No table on page 25 i

['500412.pdf']

...

[]
No table on page 89 is detected


[]
No table on page 90 is detected
[]
No table on page 91 is detected
[]
No table on page 92 is detected
['Table 2 Summary Of Pipeline Sections Requiring Further Assessment']
[]
[]
No table on page 95 is detected
[]
No table on page 96 is detected
Duration: 0:04:02.518349


[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
No table on page 24 is detected
[]
No table on page 25 is detected
[]
No table on page 26 is detected
[]
No table on page 27 is detected
[]
No table on page 28 is detected
[]
No table on page 29 is det

[]

In [29]:
hearing = 'Application for the Keystone XL Pipeline'
keystoneXL = df[df['Application title short'] == hearing].reset_index(drop = True)
keystoneXL.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\keystoneXL')
# Change the dataframe name accordingly
files = list(keystoneXL['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, keystoneXL) 

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,7,Table 1-1. For An Outline Of Where Requirement...,1.0,549189.pdf,Application for the Keystone XL Pipeline,Table
1,7,Table 1-1 Concordance Table With Neb Filing Ma...,1.0,549189.pdf,Application for the Keystone XL Pipeline,Table
2,8,Table 1-1 Concordance Table With Neb Filing Ma...,1.0,549189.pdf,Application for the Keystone XL Pipeline,Table
3,9,Table 1-1 Concordance Table With Neb Filing Ma...,1.0,549189.pdf,Application for the Keystone XL Pipeline,Table
4,10,Table 1-1 Concordance Table With Neb Filing Ma...,1.0,549189.pdf,Application for the Keystone XL Pipeline,Table


['549189.pdf', '549192.pdf', '549195.pdf', '549233.pdf', '549254.pdf', '549257.pdf', '549260.pdf', '549263.pdf', '549266.pdf', '549269.pdf', '549375.pdf', '549378.pdf', '549381.pdf', '549387.pdf', '549415.pdf', '549418.pdf', '549421.pdf', '549442.pdf', '549445.pdf']
[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
Page 3 contains an image
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
['Table 1-1. For An Outline Of Where Requirements Under The Ceaa Are Provided In This Esa, See Table 1-2.', 'Table 1-1 Concordance Table With Neb Filing Manual (2004)']
['Table 1-1 Concordance Table With Neb Filing Manual (2004) (Cont’D)']
['Table 1-1 Concordance Table With Neb Filing Manual (2004) (Cont’D)']
['Table 1-1 Concordance Table With Neb Filing Manual (2004) (Cont’D)']
['Table 1-1 Concordance Table With Neb Filing Manual (2004) (Cont’D)']
['Table 1-1 Concordance Table With Neb Filing Manual (2004) (Cont’D)']
['Table 1-1

[]
No table on page 62 is detected
['Table 4-5 Preferred Pump Station Locations']
[]
No table on page 64 is detected
[]
No table on page 65 is detected
[]
No table on page 66 is detected
[]
No table on page 67 is detected
[]
No table on page 68 is detected
[]
No table on page 69 is detected
[]
No table on page 70 is detected
[]
No table on page 71 is detected
[]
No table on page 72 is detected
[]
No table on page 73 is detected
['Table 6-1 Ownership Of Land Crossed By The Keystone Xl Pipeline']
[]
No table on page 75 is detected
[]
No table on page 76 is detected
[]
No table on page 77 is detected
[]
[]
No table on page 79 is detected
[]
No table on page 80 is detected
['Table 6-2 Sara-Listed Species Potentially Found In The Project Area']
['Table 6-2 Sara-Listed Species Potentially Found In The Project Area(Cont’D)']
[]
No table on page 83 is detected
[]
No table on page 84 is detected
[]
No table on page 85 is detected
[]
No table on page 86 is detected
['Table 7-1 Interactions Of Th

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
['Table 9-1 Interaction Of The Project With Vegetation And Wetlands']
['Table 9-1 Interaction Of The Project With Vegetation And Wetlands (Cont’D)']
['Table 9-2 Potential Environmental Effects Of Project On Vegetation Andwetlands']
[]
No table on page 6 is detected
['Table 9-3. Table 9-3 Measurable Parameters For Vegetation And Wetlands', 'Table 9-3 Measurable Parameters For Vegetation And Wetlands']
['Table 9-3 Measurable Parameters For Vegetation And Wetlands (Cont’D)']
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
['Table 9-4 Areas Of Alberta Environmentally Significant Areas Andsaskatchewan Tracts Of Native Prairie Af

...

[]
No table on page 65 is detected


[]
No table on page 66 is detected
[]
Page 67 contains an image
[]
No table on page 68 is detected
Duration: 0:03:26.552029


[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
No table on page 16 is detected
[]
[]
[]
[]
[]
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
No table on page 24 is detected
[]
No table on page 25 is detected
[]
No table on page 26 is detected
[]
No table on page 27 is detected
[]
No table on page 28 is detected
[]
No table on page 29 is detected
[]
['Table C-1 Summary Of Recommended Mitigation Measures For The Red Deerriver Contingency Crossing']
[]
No table on page 32 is detected
[]
[]
No table on page 34 is detected
[]
No table on page 35 is detected
[]
No table on page 36 is detected
[]
No table on page 37 is detected
['Table C-2 Contingency Watercourse Crossing Methods']
[]
No table on page 39 is detected
['Table C-3 Criteria For Determining The Zone Of Influence Downstream Fromwatercourse Crossings']
[]
No table on page 41 is detected
[]
No table on page 42 is de

[]

In [30]:
hearing = 'Application for the Alberta Clipper Expansion Project'
alberta_clipper = df[df['Application title short'] == hearing].reset_index(drop = True)
alberta_clipper.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\alberta_clipper')
# Change the dataframe name accordingly
files = list(alberta_clipper['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, alberta_clipper) 

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,24,Table 5A-15 Historical Mean Monthly Streamflo...,1.00,464744.pdf,Application for the Alberta Clipper Expansion ...,Table
1,30,Table 1.1 Summary Of Recent Environmental And ...,0.75,464744.pdf,Application for the Alberta Clipper Expansion ...,Table
2,31,Table 1.2 Summary Of Recent Supporting Studies...,0.75,464744.pdf,Application for the Alberta Clipper Expansion ...,Table
3,32,Table 1.2 Summary Of Recent Supporting Studies...,0.35,464744.pdf,Application for the Alberta Clipper Expansion ...,Table
4,33,Table 1.2 Summary Of Recent Supporting Studies...,0.35,464744.pdf,Application for the Alberta Clipper Expansion ...,Table


['464744.pdf', '464747.pdf', '464750.pdf', '464753.pdf', '464800.pdf', '464803.pdf', '464806.pdf', '464812.pdf', '464815.pdf', '464818.pdf', '464821.pdf', '464824.pdf', '464966.pdf', '464969.pdf', '465077.pdf']
[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 is de

[]

[]
No table on page 1 is detected
['Table 5.1 Physiographic Regions Traversed By The Proposed Pipeline Route In Alberta']
['Table 5.2 Physiographic Regions Traversed By  The Proposed Pipeline Route In Saskatchewan']
['Table 5.3 Physiographic Regions Traversed By The Proposed Pipeline Route In Manitoba', 'Table 5.4 Geologic Formations Underlying The Proposed Pipeline Route In Alberta']
['Table 5.5 Geologic Formations Underlying The Proposed Pipeline Route In Saskatchewan']
['Table 5.5 Geologic Formations Underlying The Proposed Pipeline Route In Saskatchewan']
['Table 5.6 Geologic Formations Underlying The Proposed Pipeline Route In Manitoba']
['Table 5.7 Surficial Geology Traversed By The Proposed Pipeline Route In Alberta']
['Table 5.7 Surficial Geology Traversed By The Proposed Pipeline Route In Alberta']
['Table 5.8 Surficial Geology Traversed By The Proposed Pipeline Route In Saskatchewan']
['Table 5.8 Surficial Geology Traversed By The Proposed Pipeline Route In Saskatchewan', 'Ta

[]

...

[]

['Tables 6.2, 6.3, 6.10 And 6.12 Of This Esa Outline Measures That Will Be Taken To Avoid Or Minimize Effects On The Environment For Every Phase Of The Project’S Construction Which Includes Surveying; Clearing; [...]']
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
Duration: 0:00:20.803672


[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
['Table 1. Total Estimated Indirect Ghg Emissions Contribution Due To The Electric Power Consumption At The Maximum System Capacity Of 450 Kbpd, With All Heavy Linefill (350 Cst And 11.9° Api Gravity Crude) For The [...]']
file 465077.pdf failed to open
Duration: 0:00:06.247221


['465077.pdf']

In [31]:
hearing = 'Application for Line 13 Transfer, Line 13 Reversal and Capacity Replacement for the Southern Lights Project'
southern_lights = df[df['Application title short'] == hearing].reset_index(drop = True)
southern_lights.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\southern_lights')
# Change the dataframe name accordingly
files = list(southern_lights['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, southern_lights) 

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,3,Table 3.1 Consultation Activities With Federal...,0.75,456043.pdf,"Application for Line 13 Transfer, Line 13 Reve...",Table
1,4,Table 3.1 Consultation Activities With Federal...,0.35,456043.pdf,"Application for Line 13 Transfer, Line 13 Reve...",Table
2,5,Table 3.2 Consultation Activities With Provinc...,0.75,456043.pdf,"Application for Line 13 Transfer, Line 13 Reve...",Table
3,6,Table 3.2 Consultation Activities With Provinc...,0.35,456043.pdf,"Application for Line 13 Transfer, Line 13 Reve...",Table
4,7,Table 3.2 Consultation Activities With Provinc...,0.35,456043.pdf,"Application for Line 13 Transfer, Line 13 Reve...",Table


['456043.pdf', '456046.pdf', '456049.pdf', '456220.pdf', '456223.pdf', '456226.pdf', '456229.pdf', '456238.pdf', '456241.pdf', '456244.pdf', '456247.pdf', '456253.pdf', '456316.pdf', '456319.pdf']
[]
No table on page 1 is detected
[]
No table on page 2 is detected
['Table 3.1 Consultation Activities With Federal Authorities']
['Table 3.1 Consultation Activities With Federal Authorities']
['Table 3.2 Consultation Activities With Provincial Agencies']
['Table 3.2 Consultation Activities With Provincial Agencies']
['Table 3.2 Consultation Activities With Provincial Agencies']
['Table 3.2 Consultation Activities With Provincial Agencies']
['Table 3.2 Consultation Activities With Provincial Agencies']
['Table 3.2 Consultation Activities With Provincial Agencies']
['Table 3.2 Consultation Activities With Provincial Agencies']
['Table 3.2 Consultation Activities With Provincial Agencies']
['Table 3.2 Consultation Activities With Provincial Agencies']
['Table 3.2 Consultation Activities With P

[]

[]
No table on page 1 is detected
['Table 4.1 Comparison Of Souris River Area Route Alternates']
[]
No table on page 3 is detected
[]
Page 4 contains an image
[]
Page 5 contains an image
[]
[]
[]
Page 8 contains an image
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
Duration: 0:00:32.223988


[]

...

[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]


No table on page 24 is detected
[]
No table on page 25 is detected


[]
No table on page 26 is detected
[]
No table on page 27 is detected


[]
No table on page 28 is detected
[]
No table on page 29 is detected
[]
No table on page 30 is detected
Duration: 0:00:30.677842


[]

In [32]:
hearing = 'Application for the Brunswick Pipeline Project'
brunswick = df[df['Application title short'] == hearing].reset_index(drop = True)
brunswick.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\brunswick')
# Change the dataframe name accordingly
files = list(brunswick['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, brunswick) 

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,31,Table 1.4.1 Legislation That May Be Applicable...,1.0,408937.pdf,Application for the Brunswick Pipeline Project,Table
1,32,Table 1.4.1 Legislation That May Be Applicable...,1.0,408937.pdf,Application for the Brunswick Pipeline Project,Table
2,35,Table 2.2.1 Constraints Class 1 Class 2 Class 3,1.0,408937.pdf,Application for the Brunswick Pipeline Project,Table
3,36,Table 2.2.1 Constraints Class 1 Class 2 Class 3,1.0,408937.pdf,Application for the Brunswick Pipeline Project,Table
4,47,Table 2.2.2 Length Of The Three Corridor Alter...,1.0,408937.pdf,Application for the Brunswick Pipeline Project,Table


['408937.pdf']
[]
No table on page 1 is detected


[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
No table on page 24 is detected
[]
No table on page 25 is detected
[]
No table on page 26 is detected
[]
No table on page 27 is detected
[]
No table on page 28 is detected
[]
No table on page 29 is detected
[]
No table on page 30 is de

['408937.pdf']

In [ ]:
hearing = 'Application for the Construction of North Corridor Expansion Project'
north_corridor = df[df['Application title short'] == hearing].reset_index(drop = True)
north_corridor.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\north_corridor')
# Change the dataframe name accordingly
files = list(north_corridor['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, north_corridor) 

In [33]:
hearing = 'Application for the Construction of Edson Mainline Expansion Project'
edson_mainline = df[df['Application title short'] == hearing].reset_index(drop = True)
edson_mainline.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\edson_mainline')
# Change the dataframe name accordingly
files = list(edson_mainline['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, edson_mainline) 

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,5,Table H-1 Dispositions In The Elk River Sectio...,1.0,3762095.pdf,Application for the Construction of Edson Main...,Table
1,6,Table H-1 Dispositions In The Elk River Sectio...,1.0,3762095.pdf,Application for the Construction of Edson Main...,Table
2,7,Table H-1 Dispositions In The Elk River Sectio...,1.0,3762095.pdf,Application for the Construction of Edson Main...,Table
3,8,Table H-1 Dispositions In The Elk River Sectio...,1.0,3762095.pdf,Application for the Construction of Edson Main...,Table
4,9,Table H-1 Dispositions In The Elk River Sectio...,1.0,3762095.pdf,Application for the Construction of Edson Main...,Table


['3762095.pdf', '3762428.pdf', '3762430.pdf', '3762512.pdf', '3762513.pdf', '3762514.pdf', '3762635.pdf', '3767166.pdf', '3767167.pdf', '3767168.pdf', '3767244.pdf', '3767349.pdf', '3767350.pdf']
[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
['Table H-1 Dispositions In The Elk River Section Local Assessment Area']
['Table H-1 Dispositions In The Elk River Section Local Assessment Area']
['Table H-1 Dispositions In The Elk River Section Local Assessment Area']
['Table H-1 Dispositions In The Elk River Section Local Assessment Area']
['Table H-1 Dispositions In The Elk River Section Local Assessment Area']
['Table H-1 Dispositions In The Elk River Section Local Assessment Area']
['Table H-1 Dispositions In The Elk River Section Local Assessment Area']
['Table H-1 Dispositions In The Elk River Section Local Assessment Area']
['Table H-1 Dispositions In The Elk River Section Local Assessment Area']
['

[]

[]
No table on page 1 is detected


[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
No table on page 24 is detected
[]
No table on page 25 is detected
[]
No table on page 26 is detected
[]
No table on page 27 is detected
[]
No table on page 28 is detected
[]
No table on page 29 is detected
[]
No table on page 30 is de

...

[]
Page 34 contains an image
[]
No table on page 35 is detected
[]
No table on page 36 is detected
[]
No table on page 37 is detected
[]
No table on page 38 is detected
[]
No table on page 39 is detected
[]
No table on page 40 is detected
[]
No table on page 41 is detected
[]
No table on page 42 is detected
[]
No table on page 43 is detected
[]
No table on page 44 is detected
[]
No table on page 45 is detected
[]
No table on page 46 is detected
[]
No table on page 47 is detected
[]
No table on page 48 is detected
['Table A-1 Substrate Classifications For Stream Bed And Banks', 'Table A-2 Embeddedness Code Class Description', 'Table A-3 Bank Stability']
['Table A-4 Riparian Vegetation', 'Table A-5 Instream And Overhead Cover Types']
['Table A-6 Stream Channel Pattern', 'Table A-7 Channel Islands', 'Table A-8 Sediment Bars']
['Table A-9 Coupling Code Type Description', 'Table A-10 Confinement Code Type Description', 'Table A-11 Flow Stage']
['Table A-12 Habitat Unit Classification For Sm

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
Page 3 contains an image
[]
Page 4 contains an image
[]
Page 5 contains an image
[]
Page 6 contains an image
[]
Page 7 contains an image
[]
Page 8 contains an image
[]
Page 9 contains an image
[]
Page 10 contains an image
[]
Page 11 contains an image
[]
Page 12 contains an image
[]
Page 13 contains an image
[]
Page 14 contains an image
[]
Page 15 contains an image
[]
Page 16 contains an image
[]
Page 17 contains an image
[]
Page 18 contains an image
[]
Page 19 contains an image
[]
Page 20 contains an image
[]
Page 21 contains an image
[]
Page 22 contains an image
[]
Page 23 contains an image
[]
Page 24 contains an image
[]
Page 25 contains an image
[]
Page 26 contains an image
[]
Page 27 contains an image
[]
Page 28 contains an image
[]
Page 29 contains an image
[]
Page 30 contains an image
[]
No table on page 31 is detected
[]
[]
No table on page 33 is detected
[]
No table on page 34 is detected
[]
No table on page

[]
No table on page 36 is detected
[]
No table on page 37 is detected
[]
No table on page 38 is detected
[]
No table on page 39 is detected
[]
No table on page 40 is detected
[]
No table on page 41 is detected
[]
No table on page 42 is detected
['Figure G.2-1). An Additional Watercourse (Wcdid-045) Is Within The Project Development Area (Pda)1 But Will Not Be Crossed By The Pipeline Ditch. One Drainage Will Be Crossed By The Elk River Section; An [...]', 'Figure G.2-2). Seven Drainages Will Also Be Crossed By The Alford Creek Section (Table 4).']
No table on page 43 is detected
[]
No table on page 44 is detected
['Table 1 Watercourses Supporting Fish And Fish Habitat Crossed By The Elk River Section']
[]
No table on page 46 is detected
['Table 2 Drainages Crossed By The Elk River Section']
[]
No table on page 48 is detected
['Table 3 Watercourses Supporting Fish And Fish Habitat Crossed By The Alford Creek Section']
[]
No table on page 50 is detected
['Table 4 Drainages Crossed By The 

[]

In [34]:
hearing = 'Application for Leave to Abandon Deep Panuke Pipeline'
deep_panuke = df[df['Application title short'] == hearing].reset_index(drop = True)
deep_panuke.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\deep_panuke')
# Change the dataframe name accordingly
files = list(deep_panuke['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, deep_panuke) 

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,15,Table 2.2.1 Additional Assets,1.0,3634042.pdf,Application for Leave to Abandon Deep Panuke P...,Table
1,21,Table 2.4.1 Decommissioning Emission And Disch...,1.0,3634042.pdf,Application for Leave to Abandon Deep Panuke P...,Table
2,33,Table 5.2.1 Fisheries Advisory Committee,1.0,3634042.pdf,Application for Leave to Abandon Deep Panuke P...,Table
3,34,Table 5.3.1 Summary Of Indigenous Engagement T...,1.0,3634042.pdf,Application for Leave to Abandon Deep Panuke P...,Table
4,35,Table 5.3.1 Summary Of Indigenous Engagement T...,1.0,3634042.pdf,Application for Leave to Abandon Deep Panuke P...,Table


['3634042.pdf']
[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
Page 10 contains an image
[]
Page 11 contains an image
[]
No table on page 12 is detected


[]
[]
['Table 2.2.1 Additional Assets']


[]
Page 16 contains an image


[]
Page 17 contains an image
[]
No table on page 18 is detected
[]
[]
No table on page 20 is detected
['Table 2.4.1 Decommissioning Emission And Discharge Summary']
[]
No table on page 22 is detected
[]
No table on page 23 is detected
[]
No table on page 24 is detected
[]
No table on page 25 is detected
[]
No table on page 26 is detected
[]
Page 27 contains an image
[]
No table on page 28 is detected
[]
No table on page 29 is detected
[]
No table on page 30 is detected
[]
No table on page 31 is detected
[]
Page 32 contains an image
['Table 5.2.1 Fisheries Advisory Committee']
['Table 5.3.1 Summary Of Indigenous Engagement To Date']
['Table 5.3.1 Summary Of Indigenous Engagement To Date']
['Table 5.3.1 Summary Of Indigenous Engagement To Date', 'Table 5.3.2 Summary Of Stakeholder Engagement To Date']
['Table 5.3.2 Summary Of Stakeholder Engagement To Date']
['Table 5.3.2 Summary Of Stakeholder Engagement To Date', 'Table 5.4.1 Summary Of Key Issues Raised During Indigenous And Stakehold

['3634042.pdf']

In [35]:
hearing = 'Application for the Goldboro Gas Plant and 26" Gathering Pipeline Abandonment'
goldboro_abandonment = df[df['Application title short'] == hearing].reset_index(drop = True)
goldboro_abandonment.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\goldboro_abandonment')
# Change the dataframe name accordingly
files = list(goldboro_abandonment['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, goldboro_abandonment) 

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,167,Table 6-1 Vecs And Monitoring/Follow-Up Programs,1.0,3537148.pdf,"Application for the Goldboro Gas Plant and 26""...",Table
1,172,Table 7-1 Engagement Activities With Mi’Kmaq O...,1.0,3537148.pdf,"Application for the Goldboro Gas Plant and 26""...",Table
2,175,Table 7-2 Engagement Activities With New Bruns...,1.0,3537148.pdf,"Application for the Goldboro Gas Plant and 26""...",Table
3,182,"Table 9-1 Summary: Effects, Mitigation, And Si...",1.0,3537148.pdf,"Application for the Goldboro Gas Plant and 26""...",Table
4,199,Table B1 Physical And Biological Environmental...,1.0,3537148.pdf,"Application for the Goldboro Gas Plant and 26""...",Table


['3537148.pdf']
[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
[]
[]
Page 23 contains an image
[]
No table on page 24 is detected
[]
Page 25 contains an image
[]
Page 26 contains an image
[]
Page 27 contains an image
[]
Page 28 contains an image
[]
Page 29 contains an image
[]
Page 30 contains an image
[]
Page 31 contains an image
[]
Page 32 contains an image
[]
No table on page 33 

[]
Page 35 contains an image
[]
No table on page 36 is detected
[]
No table on page 37 is detected
[]
No table on page 38 is detected


[]
No table on page 39 is detected
[]
No table on page 40 is detected
[]
No table on page 41 is detected
[]
No table on page 42 is detected


[]
No table on page 43 is detected
[]
No table on page 44 is detected


...

[]
No table on page 276 is detected


[]
No table on page 277 is detected


[]
No table on page 278 is detected


[]
No table on page 279 is detected
[]
No table on page 280 is detected
[]
[]
No table on page 282 is detected
['Table\xa0A‐1 \xa0Waste\xa0Management\xa0Requirements\xa0For\xa0Specific\xa0Waste\xa0Types No. Waste\xa0Types Category Storage\xa0_\xa0Handling Transportation Recycling\xa0_\xa0Treatment\xa0_\xa0Disposal Comments\xa0_\xa0Criteria 1 Absorbent']
file 3537148.pdf failed to open
Duration: 0:28:00.061748


['3537148.pdf']

In [36]:
hearing = 'Application for the construction of the West Path Delivery Project'
west_path = df[df['Application title short'] == hearing].reset_index(drop = True)
west_path.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\west_path')
# Change the dataframe name accordingly
files = list(west_path['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, west_path) 

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,5,Table 3-5 Slope Length (L) And Gradient (S) Va...,1.0,3461055.pdf,Application for the construction of the West P...,Table
1,9,Table 2-1 Extent Of Land Use And Soil Units Al...,1.0,3461055.pdf,Application for the construction of the West P...,Table
2,12,Table 3-1 Agricultural Suitability Classes,1.0,3461055.pdf,Application for the construction of the West P...,Table
3,13,Table 3-2 Agricultural Suitability Subclasses,1.0,3461055.pdf,Application for the construction of the West P...,Table
4,14,Table 3-3 Relative Susceptibility Of Soil To W...,1.0,3461055.pdf,Application for the construction of the West P...,Table


['3461055.pdf', '3461834.pdf', '3461938.pdf', '3462375.pdf', '3464033.pdf', '3464900.pdf']
[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
['Table 3-5 Slope Length (L) And Gradient (S) Values Associated With Various Landforms Of Alberta 9']
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
['Table 2-1 Extent Of Land Use And Soil Units Along The Project Route']
[]
No table on page 10 is detected
[]
No table on page 11 is detected
['Table 3-1 Agricultural Suitability Classes']
['Table 3-2 Agricultural Suitability Subclasses']
['Table 3-3 Relative Susceptibility Of Soil To Wind Erosion Based On Soil Texture']
['Table 3-4 Relative Susceptibility To Water Erosion Based On Soil Texture', 'Table 3-5 Slope Length (L) And Gradient (S) Values Associated With Various Landforms Of Alberta']
['Table 3-6 Compaction Risk Rating Matr

[]
No table on page 30 is detected
[]
No table on page 31 is detected
['Table A-1 Site Inspection Data']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
No table on page 42 is detected
['Table C-1 Available Nutrients', 'Table C-2 Chemical And Physical Properties Of Sampled Soils']
[]
[]
[]
['Table D-1 Soil Legend For The Ngtl West Path Delivery Project – Rocky View Section']
[]
[]
[]
No table on page 50 is detected
['Table E-1. Salt Affected Soil Criteria For Different Soil Zones']
[]
[]
No table on page 53 is detected


[]
Page 54 contains an image
[]
No table on page 55 is detected
[]
No table on page 56 is detected
Duration: 0:02:54.063152


[]

...

[]
Page 48 contains an image
[]
No table on page 49 is detected
[]
[]
No table on page 51 is detected
[]
No table on page 52 is detected
[]
No table on page 53 is detected
[]
Page 54 contains an image
[]
Page 55 contains an image
[]
Page 56 contains an image
[]
Page 57 contains an image
[]
Page 58 contains an image
[]
Page 59 contains an image
[]
Page 60 contains an image
[]
Page 61 contains an image
[]
Page 62 contains an image
[]
Page 63 contains an image
[]
Page 64 contains an image
[]
Page 65 contains an image
[]
No table on page 66 is detected


[]
No table on page 67 is detected
[]
No table on page 68 is detected
[]
No table on page 69 is detected
[]
No table on page 70 is detected


[]
No table on page 71 is detected
[]
No table on page 72 is detected
[]
No table on page 73 is detected
[]
No table on page 74 is detected
[]
No table on page 75 is detected
[]
No table on page 76 is detected
[]
No table on page 77 is detected
[]
No table on page 78 is detected
[]
No table on page 79 is detected
['Table 2-1 Ambient Atmospheric Environment Objectives']
[]
No table on page 81 is detected
['Table 3-1 Estimated Ambient Background Concentrations']
[]
[]
No table on page 84 is detected
['Table 4-1 Proposed Burton Creek Compressor Station Unit Addition Stack Parameters And Emissions']
['Table 4-1 Proposed Burton Creek Compressor Station Unit Addition Stack Parameters And Emissions']
[]
['Table 4-2 Study Area Emissions']
['Table 4-3 Stack And Emission Parameters For Regional Sources']
[]
No table on page 90 is detected
[]
No table on page 91 is detected
[]
No table on page 92 is detected
[]
No table on page 93 is detected
[]
Page 94 contains an image
['Table 5-1 Structures Co

[]
Page 111 contains an image
[]
No table on page 112 is detected
[]
[]
No table on page 114 is detected
[]
No table on page 115 is detected
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
No table on page 128 is detected
Duration: 0:11:34.173093


[]

In [37]:
hearing = 'Application for the Spruce Ridge Program'
spruce_ridge = df[df['Application title short'] == hearing].reset_index(drop = True)
spruce_ridge.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\spruce_ridge')
# Change the dataframe name accordingly
files = list(spruce_ridge['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, spruce_ridge) 

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,6,Table 3-1 Species Of Management Concern Known ...,1.0,3334565.pdf,Application for the Spruce Ridge Program,Table
1,7,Table 3-1 Species Of Management Concern Known ...,1.0,3334565.pdf,Application for the Spruce Ridge Program,Table
2,8,Table 3-1 Species Of Management Concern Known ...,1.0,3334565.pdf,Application for the Spruce Ridge Program,Table
3,9,Table 3-1 Species Of Management Concern Known ...,1.0,3334565.pdf,Application for the Spruce Ridge Program,Table
4,10,Table 3-1 Species Of Management Concern Known ...,1.0,3334565.pdf,Application for the Spruce Ridge Program,Table


['3334565.pdf', '3334566.pdf', '3335332.pdf', '3335553.pdf', '3336547.pdf', '3337530.pdf', '3340309.pdf', '3340704.pdf', '3340705.pdf', '3340706.pdf', '3340913.pdf', '3341203.pdf', '3341405.pdf', '3341517.pdf', '3341605.pdf', '3341606.pdf', '3341938.pdf', '3342531.pdf']
[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
[]
No table on page 5 is detected
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Stu

[]
['Table 4-4 Songbird Species Detected During Breeding Bird Surveys']
['Table 4-4 Songbird Species Detected During Breeding Bird Surveys']
['Table 4-4 Songbird Species Detected During Breeding Bird Surveys']
[]
No table on page 42 is detected
['Table 4-5 Summary Of Amphibians Detected During Amphibian Surveys']
['Table 4-6 Summary Of Bat Detections']
['Table 4-7 Wildlife Habitat Feature Survey Results Within The Chetwynd Loop Local Study Area']
['Table 4-7 Wildlife Habitat Feature Survey Results Within The Chetwynd Loop Local Study Area']
['Table 4-7 Wildlife Habitat Feature Survey Results Within The Chetwynd Loop Local Study Area']
['Table 4-7 Wildlife Habitat Feature Survey Results Within The Chetwynd Loop Local Study Area']
['Table 4-8 Description Of Remote Camera Locations']
['Table 4-10 Summary Of Remote Camera Detections']
['Table 4-10 Summary Of Remote Camera Detections']
[]
No table on page 52 is detected
[]
No table on page 53 is detected
[]
No table on page 54 is detected
[

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
[]
No table on page 11 is detected
['Table 3-1 Regulatory Criteria Employed For The Health Assessment']
['Table 3-2 Criteria From Other Jurisdictions Employed For The Critical Levels Assessment']
[]
No table on page 14 is detected
['Table 3-3 Summary Of Baseline Monitoring Stations', 'Table 3-4 Summary Of Baseline Air Quality For The Health Assessment']
['Table 3-4 Summary Of Baseline Air Quality For The Health Assessment']
['Table 3-5 Summary Of Baseline Air Quality For The Critical Levels Assessment']
['Table 3-6 Base Case Emission Inventory Summary']
['Table 3-8 Project-Alone Emissions Summary', 'Table 3-9 Emission Inventory Summary']
[]
No table on page 20 is detected
[]
No tabl

...

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
[]
No table on page 6 is detected
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
['Table 3-2 Summary Of Common Nighthawk Aru Survey Effort']
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
['Table 2-1 Residential Dwellings Locations']
[]
[]
[]
No table on page 7 is detected
[]
No table on page 8 is detected
['Table 3-1 Calculated Daytime And Night Time Permissible Sound Levels']
[]
No table on page 10 is detected
['Table 4-1 Acoustic Modelling Parameters']
['Table 5-1 Cs2 Sound Power Levels For Existing Facility']
['Table 5-1 Cs2 Sound Power Levels For Existing Facility']
['Table 5-2 Scenarios 1, 2, And 3 Model Results', 'Table 5-3 Low Frequency Noise Analysis']
['Table 5-4 Predicted Noise Levels From Pembina Facility Operation']
['Table 5-5 Predicted Baseline Sound Levels – Scenario 1', 'Table 5-6 Predicted Modified Baseline Case Sound Levels']
[]
['Table 6-1 Project Case Sound Power Levels']
['Table 6–2 Project Case Modelling Results']
[]
['Table 6-3 Low Frequency Noise Assessment (Bc Ogc)', 'Table 6-4 Low Frequency Noise Assessment (Health Canada)']
[]
No table on pag

[]

In [38]:
hearing = 'Application for the Spruce Ridge Program'
spruce_ridge = df[df['Application title short'] == hearing].reset_index(drop = True)
spruce_ridge.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\spruce_ridge')
# Change the dataframe name accordingly
files = list(spruce_ridge['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, spruce_ridge) 

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,6,Table 3-1 Species Of Management Concern Known ...,1.0,3334565.pdf,Application for the Spruce Ridge Program,Table
1,7,Table 3-1 Species Of Management Concern Known ...,1.0,3334565.pdf,Application for the Spruce Ridge Program,Table
2,8,Table 3-1 Species Of Management Concern Known ...,1.0,3334565.pdf,Application for the Spruce Ridge Program,Table
3,9,Table 3-1 Species Of Management Concern Known ...,1.0,3334565.pdf,Application for the Spruce Ridge Program,Table
4,10,Table 3-1 Species Of Management Concern Known ...,1.0,3334565.pdf,Application for the Spruce Ridge Program,Table


['3334565.pdf', '3334566.pdf', '3335332.pdf', '3335553.pdf', '3336547.pdf', '3337530.pdf', '3340309.pdf', '3340704.pdf', '3340705.pdf', '3340706.pdf', '3340913.pdf', '3341203.pdf', '3341405.pdf', '3341517.pdf', '3341605.pdf', '3341606.pdf', '3341938.pdf', '3342531.pdf']
[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
[]
No table on page 5 is detected
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Stu

[]
['Table 4-4 Songbird Species Detected During Breeding Bird Surveys']
['Table 4-4 Songbird Species Detected During Breeding Bird Surveys']
['Table 4-4 Songbird Species Detected During Breeding Bird Surveys']
[]
No table on page 42 is detected
['Table 4-5 Summary Of Amphibians Detected During Amphibian Surveys']
['Table 4-6 Summary Of Bat Detections']
['Table 4-7 Wildlife Habitat Feature Survey Results Within The Chetwynd Loop Local Study Area']
['Table 4-7 Wildlife Habitat Feature Survey Results Within The Chetwynd Loop Local Study Area']
['Table 4-7 Wildlife Habitat Feature Survey Results Within The Chetwynd Loop Local Study Area']
['Table 4-7 Wildlife Habitat Feature Survey Results Within The Chetwynd Loop Local Study Area']
['Table 4-8 Description Of Remote Camera Locations']
['Table 4-10 Summary Of Remote Camera Detections']
['Table 4-10 Summary Of Remote Camera Detections']
[]
No table on page 52 is detected
[]
No table on page 53 is detected
[]
No table on page 54 is detected
[

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
No table on page 9 is detected
[]
[]
No table on page 11 is detected
['Table 3-1 Regulatory Criteria Employed For The Health Assessment']
['Table 3-2 Criteria From Other Jurisdictions Employed For The Critical Levels Assessment']
[]
No table on page 14 is detected
['Table 3-3 Summary Of Baseline Monitoring Stations', 'Table 3-4 Summary Of Baseline Air Quality For The Health Assessment']
['Table 3-4 Summary Of Baseline Air Quality For The Health Assessment']
['Table 3-5 Summary Of Baseline Air Quality For The Critical Levels Assessment']
['Table 3-6 Base Case Emission Inventory Summary']
['Table 3-8 Project-Alone Emissions Summary', 'Table 3-9 Emission Inventory Summary']
[]
No table on page 20 is detected
[]
No tabl

...

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
[]
No table on page 6 is detected
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
['Table 3-1 Species Of Management Concern Known Or Likely To Occur Within The Regional Study Area']
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
['Table 3-2 Summary Of Common Nighthawk Aru Survey Effort']
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
['Table 2-1 Residential Dwellings Locations']
[]
[]
[]
No table on page 7 is detected
[]
No table on page 8 is detected
['Table 3-1 Calculated Daytime And Night Time Permissible Sound Levels']
[]
No table on page 10 is detected
['Table 4-1 Acoustic Modelling Parameters']
['Table 5-1 Cs2 Sound Power Levels For Existing Facility']
['Table 5-1 Cs2 Sound Power Levels For Existing Facility']
['Table 5-2 Scenarios 1, 2, And 3 Model Results', 'Table 5-3 Low Frequency Noise Analysis']
['Table 5-4 Predicted Noise Levels From Pembina Facility Operation']
['Table 5-5 Predicted Baseline Sound Levels – Scenario 1', 'Table 5-6 Predicted Modified Baseline Case Sound Levels']
[]
['Table 6-1 Project Case Sound Power Levels']
['Table 6–2 Project Case Modelling Results']
[]
['Table 6-3 Low Frequency Noise Assessment (Bc Ogc)', 'Table 6-4 Low Frequency Noise Assessment (Health Canada)']
[]
No table on pag

[]

In [ ]:
hearing = 'Applications for Energy East, Asset Transfer and Eastern Mainline'
energy_east = df[df['Application title short'] == hearing].reset_index(drop = True)
energy_east.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\energy_east')
# Change the dataframe name accordingly
files = list(energy_east['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, energy_east) 

,page_number,final_table_title,categories,DataID_pdf,Application title short,Category
0,33,Table 4-1 Summary Of Pipeline Re-Routes On Qué...,1.0,2967550.pdf,"Applications for Energy East, Asset Transfer a...",Table
1,34,Table 4-1 Summary Of Pipeline Re-Routes On Qué...,1.0,2967550.pdf,"Applications for Energy East, Asset Transfer a...",Table
2,35,Table 4-1 Summary Of Pipeline Re-Routes On Qué...,1.0,2967550.pdf,"Applications for Energy East, Asset Transfer a...",Table
3,40,Table 4-2 Summary Of Pipeline Re-Routes On Qué...,1.0,2967550.pdf,"Applications for Energy East, Asset Transfer a...",Table
4,41,Table 4-2 Summary Of Pipeline Re-Routes On Qué...,1.0,2967550.pdf,"Applications for Energy East, Asset Transfer a...",Table


['2967550.pdf', '2967552.pdf', '2967555.pdf', '2967560.pdf', '2967561.pdf', '2967562.pdf', '2967563.pdf', '2967565.pdf', '2967566.pdf', '2967567.pdf', '2967568.pdf', '2967569.pdf', '2967572.pdf', '2967637.pdf', '2967638.pdf', '2967639.pdf', '2967644.pdf', '2967645.pdf', '2967646.pdf', '2967649.pdf', '2967650.pdf', '2967651.pdf', '2967652.pdf', '2967653.pdf', '2967656.pdf', '2967660.pdf', '2967662.pdf', '2967664.pdf', '2967665.pdf', '2967666.pdf', '2967667.pdf', '2967668.pdf', '2967671.pdf', '2967672.pdf', '2967741.pdf', '2967742.pdf', '2967743.pdf', '2967746.pdf', '2967749.pdf', '2967750.pdf', '2967751.pdf', '2967754.pdf', '2967757.pdf', '2967758.pdf', '2967760.pdf', '2967761.pdf', '2967762.pdf', '2967767.pdf', '2967768.pdf', '2967769.pdf', '2967772.pdf', '2967773.pdf', '2967840.pdf', '2967844.pdf', '2967845.pdf', '2967846.pdf', '2967847.pdf', '2967850.pdf', '2967854.pdf', '2967855.pdf', '2967856.pdf', '2967857.pdf', '2967858.pdf', '2967861.pdf', '2967862.pdf', '2967863.pdf', '2967864.

[]
No table on page 93 is detected
[]
No table on page 94 is detected
[]
No table on page 95 is detected
[]
No table on page 96 is detected
[]
Page 97 contains an image
[]
No table on page 98 is detected
[]
No table on page 99 is detected
[]
No table on page 100 is detected
[]
Page 101 contains an image
[]
No table on page 102 is detected
[]
No table on page 103 is detected
[]
No table on page 104 is detected
[]
Page 105 contains an image
[]
No table on page 106 is detected
[]
No table on page 107 is detected
[]
No table on page 108 is detected
[]
No table on page 109 is detected
[]
No table on page 110 is detected
[]
Page 111 contains an image
[]
No table on page 112 is detected
[]
No table on page 113 is detected
[]
No table on page 114 is detected
[]
Page 115 contains an image
[]
No table on page 116 is detected
[]
No table on page 117 is detected
[]
No table on page 118 is detected
[]
Page 119 contains an image
[]
No table on page 120 is detected
[]
No table on page 121 is detected

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
No table on page 5 is detected
[]
No table on page 6 is detected
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
[]
No table on page 10 is detected
[]
No table on page 11 is detected
[]
No table on page 12 is detected
[]
No table on page 13 is detected
['Table 6-1 Recreational Fisheries And Supporting Species In The Raa']
['Table 6-1 Recreational Fisheries And Supporting Species In The Raa', 'Table 6B-1. Of The 14 Watercourses Identified As Having Potential To Contain Fish Habitat Along The New Pipeline,']
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
['Table 6-2 Potential Effects, Key Indicators And Measurable Parameters For Fish And Fish Habitat']
['Table 6-2 Potential Effects, Key Indicators And Measurable Parameters For Fish And Fish Habitat']
['Table 6-3 Potential Ef

...

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
['Table 3-1 Concordance With Neb Filing Manual, 2014-01 (Neb 2014)']
['Table 3-1 Concordance With Neb Filing Manual, 2014-01 (Neb 2014)']
['Table 3-1 Concordance With Neb Filing Manual, 2014-01 (Neb 2014)']
['Table 3-1 Concordance With Neb Filing Manual, 2014-01 (Neb 2014)']
['Table 3-1 Concordance With Neb Filing Manual, 2014-01 (Neb 2014)']
['Table 3-1 Concordance With Neb Filing Manual, 2014-01 (Neb 2014)']
['Table 3-1 Concordance With Neb Filing Manual, 2014-01 (Neb 2014)']
['Table 3-1 Concordance With Neb Filing Manual, 2014-01 (Neb 2014)']
['Table 3-1 Concordance With Neb Filing Manual, 2014-01 (Neb 2014)']
['Table 3-1 Concordance With Neb Filing Manual, 2014-01 (Neb 2014)']
['Table 3-1 Concordance With Neb Filing Manual, 2014-01 (Neb 2014)']
['Table 3-1 Concordance With Neb Filing Manual, 2014-01 (Neb 2014)']
['Table 3-1 Concordance With Neb Filing Manual, 2014-01 (Neb 2014)']
[

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
[]
No table on page 4 is detected
[]
[]
[]
No table on page 7 is detected
[]
Page 8 contains an image
[]
No table on page 9 is detected
[]
Page 10 contains an image
['Table 5-1 Measurable Parameters For Groundwater Resources']
['Table 5-2 Potential Effects On Groundwater Resources']
[]
No table on page 13 is detected
['Table 5-3 Recommended Mitigation Measures For Groundwater Resources']
['Table 5-4 Effects Classification Criteria - Groundwater Resources']
['Table 5-4 Effects Classification Criteria - Groundwater Resources']
[]
No table on page 17 is detected
[]
No table on page 18 is detected
['Table 5-5 Residual Effects On Groundwater Resources']
['Table 5-5 Residual Effects On Groundwater Resources']
['Table 5-5 Residual Effects On Groundwater Resources']
No table on page 21 is detected
[]
No table on page 22 is detected
['Table 5-6 Potential Cumulative Effects On Groundwater Resour

[]

[]
No table on page 1 is detected
[]
No table on page 2 is detected
[]
No table on page 3 is detected
['Table 4-1 Flow Volumes And Discharges – Cromer Lateral']
[]
['Table 4-2 Flow Volumes And Discharges – Assiniboine River At Portage La Prairie']
[]
No table on page 7 is detected
[]
No table on page 8 is detected
[]
[]
[]
[]
No table on page 12 is detected
[]
No table on page 13 is detected
[]
No table on page 14 is detected
[]
No table on page 15 is detected
[]
No table on page 16 is detected
[]
No table on page 17 is detected
[]
No table on page 18 is detected
[]
No table on page 19 is detected
[]
No table on page 20 is detected
[]
No table on page 21 is detected


In [ ]:
hearing = 'Application for the Wyndwood Pipeline Expansion Project',
wyndwood = df[df['Application title short'] == hearing].reset_index(drop = True)
wyndwood.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\wyndwood')
# Change the dataframe name accordingly
files = list(wyndwood['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, wyndwood) 

In [ ]:
hearing = 'Application for North Montney Project',
north_montney = df[df['Application title short'] == hearing].reset_index(drop = True)
north_montney.head()
# Change this folder to the path were you want the tables saved
os.chdir(r'F:\Environmental Baseline Data\Version 4 - Final\CSV\north_montney')
# Change the dataframe name accordingly
files = list(north_montney['DataID_pdf'].unique())
print(files)
for file in files:
    extract_tables(file, north_montney) 

In [ ]:
['Application for North Montney Project',
 'Application for Trans Mountain Expansion Project',
 'Application for the Wolverine River Lateral Loop Carmon Creek Section',
 'Applications for Energy East, Asset Transfer and Eastern Mainline  Eastern Mainline ESA',
 'Application for the Line 3 Replacement Program',
 'Application to Construct and Operate Ekwan Pipeline',
 'Application for the 2017 NGTL System Expansion',
 'Application for the Towerbirch Expansion Project',
 'Application for the Albersun Pipeline Asset Purchase G',